## **Imports**

In [2]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:,.2f}".format

## **Inputs**


In [3]:
np.random.seed(42)
VALUATION_DATE = "2023-01"
MONTHS_TO_FORECAST = 12
IMTT = 0.01

In [4]:
parameters = pd.read_csv("./data_templates/parameters.csv", index_col=0)
parameters

,1,2,3,4,5,6,7,8,9,10,11,12
AGENT_CONTRIBUTION_PERCENT,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10
BUSINESS_ACQUISITION_PERCENT,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
INFLATION_RATE,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02
MONTHS_TO_FORECAST,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00,12.00
PENSION_AND_STATUROTY_CONTRIBUTIONS_PERCENT,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11,0.11
TAX_RATE,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25
AGENT_COMMISSION,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
AGENT_SALARY,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00,50.00
CREDIT_OFFICER_COMMISSION,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
CREDIT_OFFICER_SALARY,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00


In [5]:
parameters.columns = pd.period_range(VALUATION_DATE,periods=int(parameters.columns[-1]), freq="M")

In [6]:
details_of_existing_assets = pd.read_csv("./data_templates/details_of_existing_assets.csv")
details_of_existing_assets

,asset,depreciation,net_book_value,remaining_useful_life,cost,useful_life_when_purchased,method
0,Vehicles,0.20,10000,5,20000,10,straight_line
1,Buildings,0.10,50000,20,80000,25,reducing_balance


In [7]:
details_of_new_assets = pd.read_csv("./data_templates/details_of_new_assets.csv")
details_of_new_assets

,asset,depreciation,net_book_value,remaining_useful_life,method,purchase_date,cost
0,Vehicle,0.25,10000,5,reducing_balance,01/06/2023,10000
1,Building,0.10,50000,20,reducing_balance,01/09/2023,50000


In [8]:
details_of_existing_long_term_borrowing = pd.read_csv("./data_templates/details_of_existing_long_term_borrowing.csv")
details_of_existing_long_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Lion Insurance,92800,0.06,60,60,straight_line
1,Zimnat Lion Insurance,50000,0.06,12,12,straight_line
2,Grandre,160000,0.06,36,36,straight_line
3,Minerva,15000,0.08,36,36,straight_line
4,Masawara,5613,0.06,60,60,straight_line
5,Zam-Cargo,364000,0.05,36,36,straight_line


In [9]:
details_of_new_long_term_borrowing = pd.read_csv("./data_templates/details_of_new_long_term_borrowing.csv")
details_of_new_long_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,36,36,01/08/2023,straight_line
1,ZAM,400000,0.25,60,60,01/09/2023,straight_line
2,NBS,500000,0.18,24,24,01/10/2023,straight_line
3,ZB,500000,0.21,18,18,01/10/2023,straight_line


In [10]:
details_of_new_short_term_borrowing = pd.read_csv("./data_templates/details_of_new_short_term_borrowing.csv")
details_of_new_short_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,12,12,01/07/2023,straight_line
1,ZAM,400000,0.25,12,12,01/07/2023,reducing_balance
2,NBS,500000,0.18,12,12,01/09/2023,straight_line
3,ZB,500000,0.21,12,12,01/10/2023,straight_line


In [11]:
details_of_existing_short_term_borrowing = pd.read_csv("./data_templates/details_of_existing_short_term_borrowing.csv")
details_of_existing_short_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Life,50000,0.14,10,12,straight_line
1,Zimnat Life,30000,0.13,12,12,straight_line
2,Grandre,100000,0.12,7,12,straight_line
3,FCB,120383,0.18,5,12,straight_line
4,Nurture,500000,0.25,8,12,straight_line
5,NMB,450000,0.22,11,12,straight_line
6,Beramasamba,30000,0.13,3,12,straight_line
7,Dandemutande,25000,0.12,7,12,reducing_balance


In [12]:
static_inputs_income_statement = pd.read_csv("./data_templates/static_inputs_income_statement.csv", index_col=0)
static_inputs_income_statement

,CILL,Bonus Provision,Retrenchments,Staff Welfare,Donations,Legal Fees,Strategic Expenses,Rental Income,Interest Received,Dividends Received,Realised Gains On Disposal Of Inv Properties,Exchange Gains/(Losses),Admin Fees,Fair Value Adjustments,Realised Gains On Disposal Of Equities,2% Taxation,Third Party
Jan-23,13152,14297,19268,15463,16278,0,0,0,0,0,0,0,0,0,0,0,0
Feb-23,11585,10995,14887,12027,18392,0,0,0,0,0,0,0,0,0,0,0,0
Mar-23,13943,17629,14859,12695,13104,0,0,0,0,0,0,0,0,0,0,0,0
Apr-23,17555,19467,16331,19687,17215,0,0,0,0,0,0,0,0,0,0,0,0
May-23,13073,11016,18571,15258,12454,0,0,0,0,0,0,0,0,0,0,0,0
Jun-23,11021,17869,18684,15618,18996,0,0,0,0,0,0,0,0,0,0,0,0
Jul-23,13843,16439,17208,16736,12731,0,0,0,0,0,0,0,0,0,0,0,0
Aug-23,17989,17892,15276,10391,18154,0,0,0,0,0,0,0,0,0,0,0,0
Sep-23,19692,16863,12062,15892,19762,0,0,0,0,0,0,0,0,0,0,0,0
Oct-23,16873,17916,10064,13561,15056,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
variable_inputs_income_statement = pd.read_csv("./data_templates/variable_inputs_income_statement.csv", index_col=0)
variable_inputs_income_statement

,Training,Travel Costs,Entertainment,Marketing Costs,Group Marketing Costs,Rental Costs,Subscriptions,Insurance,Repairs And Maintenance,Utilities,Stationery,Admin Costs,IT Costs,Masawara Mgt Fee,Fines And Penalties,Auditors Remuneration,Directors Fees,Consultancy Fees,Telephones,Cellphones,Internet,Courier,Fuel,Motor Vehicle Maintenance Costs,Bank Charges,Zimnat Grp Shared Costs Recovery Exp
Aug-22,6028,6484,6678,8157,9891,7038,6282,5663,8763,6636,9097,6733,9496,6930,8397,6986,5488,6802,8643,9090,9499,6648,7557,8777,5160,6218
Sep-22,5502,5863,8242,7625,8627,8302,5709,6998,6853,8696,9737,9931,5766,8582,7511,5146,7976,9061,9976,7056,8191,6445,5098,7869,9493,9496
Oct-22,9493,7790,9636,7729,6363,7237,9744,8445,9659,7999,5854,8510,9389,5608,6794,8219,6959,8369,7049,8170,6696,9199,7200,8234,6300,9735
Nov-22,5870,5563,6059,6597,6981,6306,9413,8743,6291,8152,8474,5202,7327,8272,5659,7911,7385,5262,8723,6682,8991,8923,7961,8791,9911,8104
Dec-22,9488,9191,6722,8060,6663,9029,8748,6495,8581,5698,6707,8255,7931,6147,7811,6734,7919,5623,9548,7255,9723,5253,9973,5956,8987,7735
Jan-23,5206,6757,8314,7693,6529,7675,9779,8304,8457,7160,7777,9218,5197,9282,6369,6843,9736,6016,8108,6154,9682,8183,9433,9067,6527,9555


In [14]:
existing_loans = pd.read_csv("./data_templates/existing_loans.csv", index_col=0)
existing_loans

,disbursement_date,maturity_date,loan_amount,frequency,loan_term_months,remaining_term_months,interest_rate,repayment_amount,outstanding_balance,admin_fee_percentage,credit_insurance_fee_percentage
loan_number,,,,,,,,,,,
40b0775c,43536,46386,2400,12,94,48,0.00,31.39,"1,351.78",0.06,0.05
f5c36555,43873,46386,4600,12,83,48,0.01,92.11,"3,246.86",0.06,0.07
75656bce,44068,46386,2400,12,77,48,0.01,47.81,"1,739.68",0.08,0.05
a7222f00,44611,46021,3800,12,47,36,0.00,83.95,"2,935.72",0.09,0.08
ad00cbf5,43747,46751,5000,12,99,60,0.00,52.09,"3,066.98",0.07,0.05
321ec75b,44137,46751,1700,12,86,60,0.00,24.25,"1,258.85",0.08,0.10
6747e1d4,43606,46021,1600,12,80,36,0.01,31.43,911.36,0.05,0.10
5131c13c,44591,46386,1000,12,60,48,0.01,20.90,835.18,0.05,0.08
a7222f00_1,43520,46021,1200,12,83,36,0.01,18.50,595.83,0.06,0.05


## **Disbursements**

In [15]:
def add_period_index(series: pd.Series, start_date: str, periods: int):
    index = pd.period_range(start=start_date, periods=periods, freq='M').strftime("%b-%Y")
    series.index = index
    return series

In [16]:
def calculate_sme_disbursements(
    sme_number_of_credit_officers: pd.Series,
    sme_average_loan_size: pd.Series,
    sme_number_of_loans_per_credit_officer: pd.Series,
):
    return (
        sme_number_of_credit_officers
        * sme_average_loan_size
        * sme_number_of_loans_per_credit_officer
    )

In [17]:
sme_disbursements = calculate_sme_disbursements(
    sme_number_of_credit_officers=parameters.loc["SME_NUMBER_OF_CREDIT_OFFICERS"],
    sme_average_loan_size=parameters.loc["SME_AVERAGE_LOAN_SIZE"],
    sme_number_of_loans_per_credit_officer=parameters.loc["SME_NUMBER_OF_LOANS_PER_CREDIT_OFFICER"],
)
sme_disbursements

2023-01     360,000.00
2023-02     432,000.00
2023-03     480,000.00
2023-04     360,000.00
2023-05     324,000.00
2023-06     378,000.00
2023-07     420,000.00
2023-08     595,000.00
2023-09     700,000.00
2023-10     990,000.00
2023-11   1,125,000.00
2023-12   1,125,000.00
Freq: M, dtype: float64

In [18]:
def calculate_consumer_ssb_disbursements(
    consumer_ssb_number_of_agents: pd.Series,
    consumer_ssb_average_loan_size: pd.Series,
    consumer_ssb_number_of_loans_per_agent: pd.Series,

):
    return (
        consumer_ssb_number_of_agents
        * consumer_ssb_average_loan_size
        * consumer_ssb_number_of_loans_per_agent
    )
    

In [19]:
consumer_ssb_disbursements = calculate_consumer_ssb_disbursements(
    consumer_ssb_number_of_agents=parameters.loc["CONSUMER_SSB_NUMBER_OF_AGENTS"],
    consumer_ssb_average_loan_size=parameters.loc["CONSUMER_SSB_AVERAGE_LOAN_SIZE"],
    consumer_ssb_number_of_loans_per_agent=parameters.loc[
        "CONSUMER_SSB_NUMBER_OF_LOANS_PER_AGENT"
    ],
)
consumer_ssb_disbursements

2023-01    48,000.00
2023-02    60,000.00
2023-03    72,000.00
2023-04    80,000.00
2023-05    96,000.00
2023-06    96,000.00
2023-07   112,000.00
2023-08   120,000.00
2023-09   120,000.00
2023-10   120,000.00
2023-11   120,000.00
2023-12   120,000.00
Freq: M, dtype: float64

In [20]:
def calculate_consumer_pvt_disbursements(
    consumer_pvt_number_of_agents: pd.Series,
    consumer_pvt_average_loan_size: pd.Series,
    consumer_pvt_number_of_loans_per_agent: pd.Series,

):
    return (
        consumer_pvt_number_of_agents
        * consumer_pvt_average_loan_size
        * consumer_pvt_number_of_loans_per_agent
    )


In [21]:
consumer_pvt_disbursements = calculate_consumer_pvt_disbursements(
    consumer_pvt_number_of_agents=parameters.loc["CONSUMER_PVT_NUMBER_OF_AGENTS"],
    consumer_pvt_average_loan_size=parameters.loc["CONSUMER_PVT_AVERAGE_LOAN_SIZE"],
    consumer_pvt_number_of_loans_per_agent=parameters.loc[
        "CONSUMER_PVT_NUMBER_OF_LOANS_PER_AGENT"
    ],
 
)
consumer_pvt_disbursements

2023-01    60,000.00
2023-02    75,000.00
2023-03   108,000.00
2023-04   120,000.00
2023-05   144,000.00
2023-06   162,000.00
2023-07   189,000.00
2023-08   202,500.00
2023-09   225,000.00
2023-10   225,000.00
2023-11   225,000.00
2023-12   225,000.00
Freq: M, dtype: float64

In [22]:
def calculate_b2b_disbursements(
        b2b_disbursements: pd.Series, start_date: str, periods: int
): 
    return add_period_index(b2b_disbursements, start_date=start_date, periods=periods)

In [23]:
b2b_disbursements = parameters.loc["B2B_DISBURSEMENTS"]

b2b_disbursements

2023-01   10,000.00
2023-02   10,000.00
2023-03   10,000.00
2023-04   10,000.00
2023-05   10,000.00
2023-06   10,000.00
2023-07   10,000.00
2023-08   10,000.00
2023-09   10,000.00
2023-10   10,000.00
2023-11   10,000.00
2023-12   10,000.00
Freq: M, Name: B2B_DISBURSEMENTS, dtype: float64

## **Interest Income and Principal Repayment**


### **Repayment Amount**


In [24]:
def calculate_repayment_amount(disbursements, monthly_interest_rate, average_loan_term):
    annuity_factor = (
        1 - ((1 + monthly_interest_rate) ** (-average_loan_term))
    ) / monthly_interest_rate
    return disbursements / annuity_factor

In [25]:
sme_monthly_repayment = calculate_repayment_amount(
    disbursements=sme_disbursements,
    monthly_interest_rate=parameters.loc['SME_INTEREST_RATE'],
    average_loan_term=parameters.loc['SME_AVERAGE_LOAN_TERM'],
)

sme_monthly_repayment


2023-01    81,451.18
2023-02    97,741.42
2023-03   108,601.58
2023-04    81,451.18
2023-05    73,306.06
2023-06    85,523.74
2023-07    95,026.38
2023-08   134,620.70
2023-09   158,377.30
2023-10   223,990.75
2023-11   254,534.94
2023-12   254,534.94
Freq: M, dtype: float64

In [26]:
consumer_ssb_monthly_repayment = calculate_repayment_amount(
    disbursements=consumer_ssb_disbursements,
    monthly_interest_rate=parameters.loc['CONSUMER_SSB_INTEREST_RATE'],
    average_loan_term=parameters.loc['CONSUMER_SSB_AVERAGE_LOAN_TERM'],
)

consumer_ssb_monthly_repayment

2023-01    8,006.34
2023-02   10,007.93
2023-03   12,009.51
2023-04   11,172.05
2023-05   13,406.46
2023-06   13,406.46
2023-07   15,640.87
2023-08   14,887.07
2023-09   14,887.07
2023-10   13,705.47
2023-11   13,705.47
2023-12   13,705.47
Freq: M, dtype: float64

In [27]:
consumer_pvt_monthly_repayment = calculate_repayment_amount(
    disbursements=consumer_pvt_disbursements,
    monthly_interest_rate=parameters.loc['CONSUMER_PVT_INTEREST_RATE'],
    average_loan_term=parameters.loc['CONSUMER_PVT_AVERAGE_LOAN_TERM'],
)

consumer_pvt_monthly_repayment

2023-01   10,007.93
2023-02   12,509.91
2023-03   18,014.27
2023-04   16,758.08
2023-05   20,109.69
2023-06   22,623.41
2023-07   26,393.97
2023-08   28,279.26
2023-09   31,421.40
2023-10   31,421.40
2023-11   31,421.40
2023-12   31,421.40
Freq: M, dtype: float64

In [28]:
b2b_monthly_repayment = calculate_repayment_amount(
    disbursements=b2b_disbursements,
    monthly_interest_rate=parameters.loc['B2B_INTEREST_RATE'],
    average_loan_term=parameters.loc['B2B_AVERAGE_LOAN_TERM'],
)

b2b_monthly_repayment

2023-01   1,627.45
2023-02   1,627.45
2023-03   1,627.45
2023-04   1,627.45
2023-05   1,627.45
2023-06   1,627.45
2023-07   1,627.45
2023-08   1,627.45
2023-09   1,627.45
2023-10   1,627.45
2023-11   1,627.45
2023-12   1,627.45
Freq: M, dtype: float64

### **Loan Schedules**


In [29]:
def generate_columns(start_date: str, period: int):
    return pd.period_range(start=start_date, periods=period, freq="M").strftime("%b-%Y")

In [30]:
def shift(df: pd.DataFrame):
    for i in range(len(df)):
        df.iloc[i] = df.iloc[i].shift(i)
    return df.fillna(0)

In [31]:
def generate_loan_schedule(
    disbursements: pd.Series,
    monthly_interest_rate: float,
    repayment_amount: pd.Series,
    months_to_project: int,
) -> dict:
    number_of_disbursements = len(disbursements)

    outstanding_at_start = np.zeros(
        (number_of_disbursements, months_to_project))
    interest = np.zeros((number_of_disbursements, months_to_project))
    capital_repayments = np.zeros((number_of_disbursements, months_to_project))
    outstanding_at_start[:, 0] = disbursements.values

    for i in range(months_to_project):
        interest[:, i] = outstanding_at_start[:, i] * monthly_interest_rate
        capital_repayments[:, i] = np.where(
            outstanding_at_start[:,
                                 i] < repayment_amount.values - interest[:, i],
            outstanding_at_start[:, i],
            repayment_amount.values - interest[:, i],
        )
        if i + 1 == months_to_project:
            break
        outstanding_at_start[:, i + 1] = np.maximum(
            outstanding_at_start[:, i] - capital_repayments[:, i], 0
        )
    date_of_disbursement = disbursements.index
    columns = generate_columns(date_of_disbursement[1], MONTHS_TO_FORECAST)
    return {
        "interest": shift(pd.DataFrame(interest, index=date_of_disbursement, columns=columns)),
        "capital_repayments": shift(pd.DataFrame(
            capital_repayments, index=date_of_disbursement, columns=columns
        )),
        "outstanding_at_start": shift(pd.DataFrame(
            outstanding_at_start, index=date_of_disbursement, columns=columns
        )),
    }

In [32]:
sme_loan_schedules = generate_loan_schedule(
    disbursements=sme_disbursements,
    monthly_interest_rate=parameters.loc['SME_INTEREST_RATE'],
    repayment_amount=sme_monthly_repayment,
    months_to_project=MONTHS_TO_FORECAST,
)
sme_loan_schedules['interest']

,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024
2023-01,"34,200.00","29,711.14","24,795.83","19,413.58","13,520.00","7,066.54",0.00,0.00,0.00,0.00,0.00,0.00
2023-02,0.00,"41,040.00","35,653.37","29,755.00","23,296.29","16,224.00","8,479.85",0.00,0.00,0.00,0.00,0.00
2023-03,0.00,0.00,"45,600.00","39,614.85","33,061.11","25,884.77","18,026.67","9,422.05",0.00,0.00,0.00,0.00
2023-04,0.00,0.00,0.00,"34,200.00","29,711.14","24,795.83","19,413.58","13,520.00","7,066.54",0.00,0.00,0.00
2023-05,0.00,0.00,0.00,0.00,"30,780.00","26,740.02","22,316.25","17,472.22","12,168.00","6,359.89",0.00,0.00
2023-06,0.00,0.00,0.00,0.00,0.00,"35,910.00","31,196.69","26,035.63","20,384.25","14,196.00","7,419.87",0.00
2023-07,0.00,0.00,0.00,0.00,0.00,0.00,"39,900.00","34,662.99","28,928.47","22,649.17","15,773.34","8,244.30"
2023-08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"56,525.00","49,105.91","40,982.00","32,086.33","22,345.56"
2023-09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"66,500.00","57,771.66","48,214.12","37,748.62"
2023-10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"94,050.00","81,705.63","68,188.54"


In [33]:
sme_loan_schedules['capital_repayments']

,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024
2023-01,"47,251.18","51,740.04","56,655.35","62,037.61","67,931.18","74,384.64",0.00,0.00,0.00,0.00,0.00,0.00
2023-02,0.00,"56,701.42","62,088.05","67,986.42","74,445.13","81,517.41","89,261.57",0.00,0.00,0.00,0.00,0.00
2023-03,0.00,0.00,"63,001.58","68,986.73","75,540.46","82,716.81","90,574.91","99,179.52",0.00,0.00,0.00,0.00
2023-04,0.00,0.00,0.00,"47,251.18","51,740.04","56,655.35","62,037.61","67,931.18","74,384.64",0.00,0.00,0.00
2023-05,0.00,0.00,0.00,0.00,"42,526.06","46,566.04","50,989.81","55,833.85","61,138.06","66,946.18",0.00,0.00
2023-06,0.00,0.00,0.00,0.00,0.00,"49,613.74","54,327.05","59,488.12","65,139.49","71,327.74","78,103.87",0.00
2023-07,0.00,0.00,0.00,0.00,0.00,0.00,"55,126.38","60,363.38","66,097.91","72,377.21","79,253.04","86,782.08"
2023-08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"78,095.70","85,514.79","93,638.70","102,534.38","112,275.14"
2023-09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"91,877.30","100,605.64","110,163.18","120,628.68"
2023-10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"129,940.75","142,285.12","155,802.21"


In [34]:
sme_loan_schedules = generate_loan_schedule(
    disbursements=sme_disbursements,
    monthly_interest_rate=parameters.loc["SME_INTEREST_RATE"],
    repayment_amount=sme_monthly_repayment,
    months_to_project=MONTHS_TO_FORECAST,
)
consumer_ssb_loan_schedules = generate_loan_schedule(
    disbursements=consumer_ssb_disbursements,
    monthly_interest_rate=parameters.loc["CONSUMER_SSB_INTEREST_RATE"],
    repayment_amount=consumer_ssb_monthly_repayment,
    months_to_project=MONTHS_TO_FORECAST,
)
b2b_loan_schedules = generate_loan_schedule(
    disbursements=b2b_disbursements,
    monthly_interest_rate=parameters.loc["B2B_INTEREST_RATE"],
    repayment_amount=b2b_monthly_repayment,
    months_to_project=MONTHS_TO_FORECAST,
)
consumer_pvt_loan_schedules = generate_loan_schedule(
    disbursements=consumer_pvt_disbursements,
    monthly_interest_rate=parameters.loc["CONSUMER_PVT_INTEREST_RATE"],
    repayment_amount=consumer_pvt_monthly_repayment,
    months_to_project=MONTHS_TO_FORECAST,
)

In [35]:
sme_loan_schedules.keys()

dict_keys(['interest', 'capital_repayments', 'outstanding_at_start'])

In [36]:
sme_loan_schedules["interest"]

,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024
2023-01,"34,200.00","29,711.14","24,795.83","19,413.58","13,520.00","7,066.54",0.00,0.00,0.00,0.00,0.00,0.00
2023-02,0.00,"41,040.00","35,653.37","29,755.00","23,296.29","16,224.00","8,479.85",0.00,0.00,0.00,0.00,0.00
2023-03,0.00,0.00,"45,600.00","39,614.85","33,061.11","25,884.77","18,026.67","9,422.05",0.00,0.00,0.00,0.00
2023-04,0.00,0.00,0.00,"34,200.00","29,711.14","24,795.83","19,413.58","13,520.00","7,066.54",0.00,0.00,0.00
2023-05,0.00,0.00,0.00,0.00,"30,780.00","26,740.02","22,316.25","17,472.22","12,168.00","6,359.89",0.00,0.00
2023-06,0.00,0.00,0.00,0.00,0.00,"35,910.00","31,196.69","26,035.63","20,384.25","14,196.00","7,419.87",0.00
2023-07,0.00,0.00,0.00,0.00,0.00,0.00,"39,900.00","34,662.99","28,928.47","22,649.17","15,773.34","8,244.30"
2023-08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"56,525.00","49,105.91","40,982.00","32,086.33","22,345.56"
2023-09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"66,500.00","57,771.66","48,214.12","37,748.62"
2023-10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"94,050.00","81,705.63","68,188.54"


In [37]:
sme_loan_schedules["capital_repayments"]

,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024
2023-01,"47,251.18","51,740.04","56,655.35","62,037.61","67,931.18","74,384.64",0.00,0.00,0.00,0.00,0.00,0.00
2023-02,0.00,"56,701.42","62,088.05","67,986.42","74,445.13","81,517.41","89,261.57",0.00,0.00,0.00,0.00,0.00
2023-03,0.00,0.00,"63,001.58","68,986.73","75,540.46","82,716.81","90,574.91","99,179.52",0.00,0.00,0.00,0.00
2023-04,0.00,0.00,0.00,"47,251.18","51,740.04","56,655.35","62,037.61","67,931.18","74,384.64",0.00,0.00,0.00
2023-05,0.00,0.00,0.00,0.00,"42,526.06","46,566.04","50,989.81","55,833.85","61,138.06","66,946.18",0.00,0.00
2023-06,0.00,0.00,0.00,0.00,0.00,"49,613.74","54,327.05","59,488.12","65,139.49","71,327.74","78,103.87",0.00
2023-07,0.00,0.00,0.00,0.00,0.00,0.00,"55,126.38","60,363.38","66,097.91","72,377.21","79,253.04","86,782.08"
2023-08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"78,095.70","85,514.79","93,638.70","102,534.38","112,275.14"
2023-09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"91,877.30","100,605.64","110,163.18","120,628.68"
2023-10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"129,940.75","142,285.12","155,802.21"


In [38]:
sme_loan_schedules["outstanding_at_start"]

,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023,Jan-2024
2023-01,"360,000.00","312,748.82","261,008.77","204,353.43","142,315.82","74,384.64",0.00,0.00,0.00,0.00,0.00,0.00
2023-02,0.00,"432,000.00","375,298.58","313,210.53","245,224.11","170,778.98","89,261.57",0.00,0.00,0.00,0.00,0.00
2023-03,0.00,0.00,"480,000.00","416,998.42","348,011.70","272,471.23","189,754.43","99,179.52",0.00,0.00,0.00,0.00
2023-04,0.00,0.00,0.00,"360,000.00","312,748.82","261,008.77","204,353.43","142,315.82","74,384.64",0.00,0.00,0.00
2023-05,0.00,0.00,0.00,0.00,"324,000.00","281,473.94","234,907.90","183,918.08","128,084.24","66,946.18",0.00,0.00
2023-06,0.00,0.00,0.00,0.00,0.00,"378,000.00","328,386.26","274,059.21","214,571.10","149,431.61","78,103.87",0.00
2023-07,0.00,0.00,0.00,0.00,0.00,0.00,"420,000.00","364,873.62","304,510.24","238,412.33","166,035.12","86,782.08"
2023-08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"595,000.00","516,904.30","431,389.50","337,750.80","235,216.42"
2023-09,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"700,000.00","608,122.70","507,517.06","397,353.88"
2023-10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"990,000.00","860,059.25","717,774.13"


## **Admin Feee**


In [39]:
average_loan_term = int(parameters.loc['CONSUMER_SSB_AVERAGE_LOAN_TERM'].mean())
average_loan_term

13

In [40]:
def calculate_admin_fee(
    disbursements: pd.Series,
    admin_fee_percentage: float,
    average_loan_term: int,
    months_to_forecast: int | pd.Series,
) -> pd.Series:
    
    average_loan_term = int(average_loan_term.mean())
    
    admin_fee = disbursements * admin_fee_percentage / average_loan_term
    admin_fee = np.repeat(admin_fee.values, average_loan_term).reshape(
        len(admin_fee), average_loan_term
    )
    admin_fee = pd.DataFrame(
        admin_fee,
        columns=generate_columns(disbursements.index[0], average_loan_term),
        index=disbursements.index,
    )
    admin_fee = admin_fee.reindex(generate_columns(disbursements.index[0], months_to_forecast),axis=1)
    return shift(admin_fee).sum()

In [41]:
sme_admin_fee = calculate_admin_fee(
    sme_disbursements,
    admin_fee_percentage=parameters.loc['SME_ADMINISTRATION_FEE'],
    average_loan_term=parameters.loc['SME_AVERAGE_LOAN_TERM'],
    months_to_forecast=MONTHS_TO_FORECAST,
)
b2b_admin_fee = calculate_admin_fee(
    b2b_disbursements,
    admin_fee_percentage=parameters.loc['B2B_ADMINISTRATION_FEE'],
    average_loan_term=parameters.loc['B2B_AVERAGE_LOAN_TERM'],
    months_to_forecast=MONTHS_TO_FORECAST,
)
consumer_ssb_admin_fee = calculate_admin_fee(
    consumer_ssb_disbursements,
    admin_fee_percentage=parameters.loc['CONSUMER_SSB_ADMINISTRATION_FEE'],
    average_loan_term=parameters.loc['CONSUMER_SSB_AVERAGE_LOAN_TERM'],
    months_to_forecast=MONTHS_TO_FORECAST,
)
consumer_pvt_admin_fee = calculate_admin_fee(
    consumer_pvt_disbursements,
   admin_fee_percentage=parameters.loc['CONSUMER_PVT_ADMINISTRATION_FEE'],
    average_loan_term=parameters.loc['CONSUMER_PVT_AVERAGE_LOAN_TERM'],
    months_to_forecast=MONTHS_TO_FORECAST,
)

In [42]:
sme_admin_fee

Jan-2023    3,000.00
Feb-2023    6,600.00
Mar-2023   10,600.00
Apr-2023   13,600.00
May-2023   16,300.00
Jun-2023   19,450.00
Jul-2023   19,950.00
Aug-2023   21,308.33
Sep-2023   23,141.67
Oct-2023   28,391.67
Nov-2023   35,066.67
Dec-2023   41,291.67
dtype: float64

In [43]:
b2b_admin_fee

Jan-2023    10.00
Feb-2023    20.00
Mar-2023    30.00
Apr-2023    40.00
May-2023    50.00
Jun-2023    60.00
Jul-2023    70.00
Aug-2023    80.00
Sep-2023    90.00
Oct-2023   100.00
Nov-2023   100.00
Dec-2023   100.00
dtype: float64

In [44]:
consumer_ssb_admin_fee

Jan-2023     184.62
Feb-2023     415.38
Mar-2023     692.31
Apr-2023   1,000.00
May-2023   1,369.23
Jun-2023   1,738.46
Jul-2023   2,169.23
Aug-2023   2,630.77
Sep-2023   3,092.31
Oct-2023   3,553.85
Nov-2023   4,015.38
Dec-2023   4,476.92
dtype: float64

In [45]:
consumer_pvt_admin_fee

Jan-2023     272.73
Feb-2023     613.64
Mar-2023   1,104.55
Apr-2023   1,650.00
May-2023   2,304.55
Jun-2023   3,040.91
Jul-2023   3,900.00
Aug-2023   4,820.45
Sep-2023   5,843.18
Oct-2023   6,865.91
Nov-2023   7,888.64
Dec-2023   8,638.64
dtype: float64

In [47]:
sme_insurance = calculate_admin_fee(
    sme_disbursements,
    admin_fee_percentage=parameters.loc['SME_CREDIT_INSURANCE_FEE'],
    average_loan_term=parameters.loc['SME_AVERAGE_LOAN_TERM'],
    months_to_forecast=MONTHS_TO_FORECAST,
)
b2b_insurance = calculate_admin_fee(
    b2b_disbursements,
    admin_fee_percentage=parameters.loc['B2B_CREDIT_INSURANCE_FEE'],
    average_loan_term=parameters.loc['B2B_AVERAGE_LOAN_TERM'],
    months_to_forecast=MONTHS_TO_FORECAST,
)
consumer_ssb_insurance = calculate_admin_fee(
    consumer_ssb_disbursements,
    admin_fee_percentage=parameters.loc['CONSUMER_SSB_CREDIT_INSURANCE_FEE'],
    average_loan_term=parameters.loc['CONSUMER_SSB_AVERAGE_LOAN_TERM'],
    months_to_forecast=MONTHS_TO_FORECAST,
)
consumer_pvt_insurance = calculate_admin_fee(
    consumer_pvt_disbursements,
   admin_fee_percentage=parameters.loc['CONSUMER_PVT_CREDIT_INSURANCE_FEE'],
    average_loan_term=parameters.loc['CONSUMER_PVT_AVERAGE_LOAN_TERM'],
    months_to_forecast=MONTHS_TO_FORECAST,
)

In [ ]:

def aggregate_interest_income(
    sme_loan_schedules, ssb_loan_schedules, b2b_loan_schedules, consumer_loan_schedules
):
    return (
        sme_loan_schedules["interest"].sum()
        + ssb_loan_schedules["interest"].sum()
        + b2b_loan_schedules["interest"].sum()
        + consumer_loan_schedules["interest"].sum()
    )

In [ ]:
interest_income_new_disbursements = aggregate_interest_income(sme_loan_schedules,consumer_ssb_loan_schedules,b2b_loan_schedules, consumer_pvt_loan_schedules)
interest_income_new_disbursements

Feb-2023    44,920.00
Mar-2023    93,811.91
Apr-2023   144,431.43
May-2023   177,009.85
Jun-2023   202,378.60
Jul-2023   226,681.27
Aug-2023   249,538.43
Sep-2023   287,795.92
Oct-2023   334,040.92
Nov-2023   403,074.16
Dec-2023   475,262.01
Jan-2024   534,975.55
dtype: float64

In [ ]:
def generate_loan_schedules_existing_loans(
    outstanding_balance: pd.Series,
    interest_rate_monthly: pd.Series,
    repayment_amount_monthly: pd.Series,
    valuation_date: str,
    months_to_project: int = 12 * 10,
):
    number_of_loans = outstanding_balance.shape[0]

    outstanding_at_start = np.zeros((number_of_loans, months_to_project))
    interest = np.zeros((number_of_loans, months_to_project))
    capital_repayment = np.zeros((number_of_loans, months_to_project))

    outstanding_at_start[:, 0] = outstanding_balance.values
    for i in range(months_to_project - 1):
        interest[:, i] = outstanding_at_start[:, i] * interest_rate_monthly
        capital_repayment[:, i] = np.where(
            outstanding_at_start[:, i] == 0,
            0,
            repayment_amount_monthly - interest[:, i],
        )
        outstanding_at_start[:, i + 1] = np.maximum(
            0, outstanding_at_start[:, i] - capital_repayment[:, i]
        )

    index = outstanding_balance.index
    columns = generate_columns(valuation_date, months_to_project)


    return {
        "interest": pd.DataFrame(interest, index=index, columns=columns),
        "capital_repayment": pd.DataFrame(
            capital_repayment, index=index, columns=columns
        ),
        "outstanding_at_start": pd.DataFrame(
            outstanding_at_start, index=index, columns=columns
        ),
    }

In [ ]:
existing_loans_schedules = generate_loan_schedules_existing_loans(
    outstanding_balance=existing_loans["outstanding_balance"],
    interest_rate_monthly=existing_loans["interest_rate"],
    repayment_amount_monthly=existing_loans["repayment_amount"],
    valuation_date=VALUATION_DATE, 
    months_to_project=MONTHS_TO_FORECAST,
)

existing_loans_schedules.keys()

dict_keys(['interest', 'capital_repayment', 'outstanding_at_start'])

In [ ]:
def aggregate_new_and_existing_loans_interest_income(
    interest_income_new_disbursements: pd.Series,
    interest_income_existing_loans: pd.Series,
):
    return interest_income_new_disbursements.add(interest_income_existing_loans, fill_value=0).reindex(generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST))

In [ ]:
total_interest_income = aggregate_new_and_existing_loans_interest_income(
    interest_income_new_disbursements=interest_income_new_disbursements,
    interest_income_existing_loans=existing_loans_schedules["interest"].sum(),
)

total_interest_income

Jan-2023       290.78
Feb-2023    45,203.97
Mar-2023    94,088.99
Apr-2023   144,701.56
May-2023   177,272.94
Jun-2023   202,634.57
Jul-2023   226,930.05
Aug-2023   249,779.94
Sep-2023   288,030.08
Oct-2023   334,267.64
Nov-2023   403,293.37
Dec-2023   475,262.01
dtype: float64

In [ ]:
def calculate_admin_fee(
    disbursements: pd.Series,
    admin_fee_percentage: float,
    average_loan_term: int,
    months_to_forecast: int | pd.Series,
) -> pd.Series:
    
    average_loan_term = int(average_loan_term.mean())
    
    admin_fee = disbursements * admin_fee_percentage / average_loan_term
    admin_fee = np.repeat(admin_fee.values, average_loan_term).reshape(
        len(admin_fee), average_loan_term
    )
    admin_fee = pd.DataFrame(
        admin_fee,
        columns=generate_columns(disbursements.index[0], average_loan_term),
        index=disbursements.index,
    )
    admin_fee = admin_fee.reindex(generate_columns(disbursements.index[0], months_to_forecast),axis=1)
    return shift(admin_fee).sum()

In [ ]:
existing_loans

,disbursement_date,maturity_date,loan_amount,frequency,loan_term_months,remaining_term_months,interest_rate,repayment_amount,outstanding_balance,admin_fee_percentage,credit_insurance_fee_percentage
loan_number,,,,,,,,,,,
40b0775c,43536,46386,2400,12,94,48,0.00,31.39,"1,351.78",0.06,0.05
f5c36555,43873,46386,4600,12,83,48,0.01,92.11,"3,246.86",0.06,0.07
75656bce,44068,46386,2400,12,77,48,0.01,47.81,"1,739.68",0.08,0.05
a7222f00,44611,46021,3800,12,47,36,0.00,83.95,"2,935.72",0.09,0.08
ad00cbf5,43747,46751,5000,12,99,60,0.00,52.09,"3,066.98",0.07,0.05
321ec75b,44137,46751,1700,12,86,60,0.00,24.25,"1,258.85",0.08,0.10
6747e1d4,43606,46021,1600,12,80,36,0.01,31.43,911.36,0.05,0.10
5131c13c,44591,46386,1000,12,60,48,0.01,20.90,835.18,0.05,0.08
a7222f00_1,43520,46021,1200,12,83,36,0.01,18.50,595.83,0.06,0.05


In [ ]:
admin_fee_existing = existing_loans.loan_amount * existing_loans.admin_fee_percentage / existing_loans.loan_term_months
admin_fee_existing

loan_number
40b0775c     1.53
f5c36555     3.33
75656bce     2.49
a7222f00     7.28
ad00cbf5     3.54
321ec75b     1.58
6747e1d4     1.00
5131c13c     0.83
a7222f00_1   0.87
1ae65220     3.73
7aaaa107     5.42
d3d56735     6.67
c71f64fd     3.75
d3d56735_1   3.54
c5f5db38     3.45
99e2b327     6.24
e66b480c     3.70
14c22098     2.21
b89a6836     5.07
dtype: float64

In [ ]:
admin_fees_existing_loans = []
for i in admin_fee_existing.index:
    temp = pd.Series(np.repeat(admin_fee_existing.loc[i], existing_loans.loc[i,'remaining_term_months']), name=i)
    admin_fees_existing_loans.append(temp)
admin_fees_existing_loans = pd.concat(admin_fees_existing_loans, axis=1).T.fillna(0)
admin_fees_existing_loans = admin_fees_existing_loans.iloc[:,:MONTHS_TO_FORECAST]
admin_fees_existing_loans.columns = generate_columns(VALUATION_DATE, period=MONTHS_TO_FORECAST)
admin_fees_existing_loans

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
40b0775c,1.53,1.53,1.53,1.53,1.53,1.53,1.53,1.53,1.53,1.53,1.53,1.53
f5c36555,3.33,3.33,3.33,3.33,3.33,3.33,3.33,3.33,3.33,3.33,3.33,3.33
75656bce,2.49,2.49,2.49,2.49,2.49,2.49,2.49,2.49,2.49,2.49,2.49,2.49
a7222f00,7.28,7.28,7.28,7.28,7.28,7.28,7.28,7.28,7.28,7.28,7.28,7.28
ad00cbf5,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54
321ec75b,1.58,1.58,1.58,1.58,1.58,1.58,1.58,1.58,1.58,1.58,1.58,1.58
6747e1d4,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
5131c13c,0.83,0.83,0.83,0.83,0.83,0.83,0.83,0.83,0.83,0.83,0.83,0.83
a7222f00_1,0.87,0.87,0.87,0.87,0.87,0.87,0.87,0.87,0.87,0.87,0.87,0.87
1ae65220,3.73,3.73,3.73,3.73,3.73,3.73,3.73,3.73,3.73,3.73,3.73,3.73


In [ ]:
def calculate_admin_fee_existing_loans(
    loan_amount: pd.Series, admin_fee_percentage: pd.Series, loan_term_months: pd.Series

    , remaining_term_months: pd.Series
):
    admin_fee_existing = loan_amount * admin_fee_percentage / loan_term_months

    admin_fees_existing_loans = []
    for i in admin_fee_existing.index:
        temp = pd.Series(
            np.repeat(
                admin_fee_existing.loc[i],
                remaining_term_months.loc[i],
            ),
            name=i,
        )
        admin_fees_existing_loans.append(temp)
    admin_fees_existing_loans = pd.concat(admin_fees_existing_loans, axis=1).T.fillna(0)
    admin_fees_existing_loans = admin_fees_existing_loans.iloc[:, :MONTHS_TO_FORECAST]
    admin_fees_existing_loans.columns = generate_columns(
        VALUATION_DATE, period=MONTHS_TO_FORECAST
    )
    admin_fees_existing_loans

    return admin_fees_existing_loans.sum()

In [ ]:
admin_fee_existing_loans = calculate_admin_fee_existing_loans(
    loan_amount=existing_loans.loan_amount,
    admin_fee_percentage=existing_loans.admin_fee_percentage,
    loan_term_months=existing_loans.loan_term_months,
    remaining_term_months=existing_loans.remaining_term_months
)

admin_fees_existing_loans

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
40b0775c,1.53,1.53,1.53,1.53,1.53,1.53,1.53,1.53,1.53,1.53,1.53,1.53
f5c36555,3.33,3.33,3.33,3.33,3.33,3.33,3.33,3.33,3.33,3.33,3.33,3.33
75656bce,2.49,2.49,2.49,2.49,2.49,2.49,2.49,2.49,2.49,2.49,2.49,2.49
a7222f00,7.28,7.28,7.28,7.28,7.28,7.28,7.28,7.28,7.28,7.28,7.28,7.28
ad00cbf5,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54,3.54
321ec75b,1.58,1.58,1.58,1.58,1.58,1.58,1.58,1.58,1.58,1.58,1.58,1.58
6747e1d4,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
5131c13c,0.83,0.83,0.83,0.83,0.83,0.83,0.83,0.83,0.83,0.83,0.83,0.83
a7222f00_1,0.87,0.87,0.87,0.87,0.87,0.87,0.87,0.87,0.87,0.87,0.87,0.87
1ae65220,3.73,3.73,3.73,3.73,3.73,3.73,3.73,3.73,3.73,3.73,3.73,3.73


In [ ]:
def aggregate_admin_fee(
    sme_admin_fee: pd.Series,
    consumer_pvt_admin_fee: pd.Series,
    consumer_ssb_admin_fee: pd.Series,
    b2b_admin_fee: pd.Series,
):
    return (
        sme_admin_fee + consumer_pvt_admin_fee + consumer_ssb_admin_fee + b2b_admin_fee
    )

In [ ]:
admin_fee_new = aggregate_admin_fee(
    sme_admin_fee,
    consumer_pvt_admin_fee,
    consumer_ssb_admin_fee,
    b2b_admin_fee,
)
admin_fee_new

Jan-2023    3,467.34
Feb-2023    7,649.02
Mar-2023   12,426.85
Apr-2023   16,290.00
May-2023   20,023.78
Jun-2023   24,289.37
Jul-2023   26,089.23
Aug-2023   28,839.56
Sep-2023   32,167.16
Oct-2023   38,911.42
Nov-2023   47,070.69
Dec-2023   54,507.23
dtype: float64

In [ ]:
admin_fee_existing_loans

Jan-2023   66.22
Feb-2023   66.22
Mar-2023   66.22
Apr-2023   66.22
May-2023   66.22
Jun-2023   66.22
Jul-2023   66.22
Aug-2023   66.22
Sep-2023   66.22
Oct-2023   66.22
Nov-2023   66.22
Dec-2023   66.22
dtype: float64

In [ ]:
def aggregate_new_and_existing_loans_admin_fee(
    admin_fee_existing: pd.Series, admin_fee_new: pd.Series
):
    return admin_fee_new.add(admin_fee_existing, fill_value=0)

In [ ]:
total_admin_fee = aggregate_new_and_existing_loans_admin_fee(
    admin_fee_existing=admin_fee_existing_loans, admin_fee_new=admin_fee_new
)
total_admin_fee

Jan-2023    3,533.56
Feb-2023    7,715.24
Mar-2023   12,493.08
Apr-2023   16,356.22
May-2023   20,090.00
Jun-2023   24,355.59
Jul-2023   26,155.45
Aug-2023   28,905.78
Sep-2023   32,233.38
Oct-2023   38,977.64
Nov-2023   47,136.91
Dec-2023   54,573.45
dtype: float64


## **Credit Insurance**

In [ ]:
def aggregate_insurance(
    sme_insurance: pd.Series,
    consumer_pvt_insurance: pd.Series,
    consumer_ssb_insurance: pd.Series,
    b2b_insurance: pd.Series,
):
    return (
        sme_insurance + consumer_pvt_insurance + consumer_ssb_insurance + b2b_insurance
    )

In [ ]:
insurance_new = aggregate_insurance(
    sme_insurance,
    consumer_pvt_insurance,
    consumer_ssb_insurance,
    b2b_insurance,
)
insurance_new

Jan-2023    1,382.94
Feb-2023    3,051.61
Mar-2023    4,958.74
Apr-2023    6,500.00
May-2023    7,989.51
Jun-2023    9,691.75
Jul-2023   10,407.69
Aug-2023   11,503.82
Sep-2023   12,830.86
Oct-2023   15,524.57
Nov-2023   18,788.28
Dec-2023   21,762.89
dtype: float64

In [ ]:
def calculate_insurance_fee_existing_loans(
    loan_amount: pd.Series, insurance_fee_percentage: pd.Series, loan_term_months: pd.Series, 
   remaining_term_months: pd.Series
):
    insurance_fee_existing = loan_amount * insurance_fee_percentage / loan_term_months

    insurance_fees_existing_loans = []
    for i in insurance_fee_existing.index:
        temp = pd.Series(
            np.repeat(
                insurance_fee_existing.loc[i],
                remaining_term_months.loc[i],
            ),
            name=i,
        )
        insurance_fees_existing_loans.append(temp)
    insurance_fees_existing_loans = pd.concat(insurance_fees_existing_loans, axis=1).T.fillna(0)
    insurance_fees_existing_loans = insurance_fees_existing_loans.iloc[:, :MONTHS_TO_FORECAST]
    insurance_fees_existing_loans.columns = generate_columns(
        VALUATION_DATE, period=MONTHS_TO_FORECAST
    )
    insurance_fees_existing_loans

    return insurance_fees_existing_loans.sum()

In [ ]:
insurance_fee_existing_loans = calculate_insurance_fee_existing_loans(
    loan_amount=existing_loans.loan_amount,
    insurance_fee_percentage=existing_loans.credit_insurance_fee_percentage,
    loan_term_months=existing_loans.loan_term_months,
)

insurance_fee_existing_loans

Jan-2023   61.55
Feb-2023   61.55
Mar-2023   61.55
Apr-2023   61.55
May-2023   61.55
Jun-2023   61.55
Jul-2023   61.55
Aug-2023   61.55
Sep-2023   61.55
Oct-2023   61.55
Nov-2023   61.55
Dec-2023   61.55
dtype: float64

In [ ]:
def aggregate_new_and_existing_loans_insurance_fee(
    insurance_fee_existing: pd.Series, insurance_fee_new: pd.Series
):
    return insurance_fee_new.add(insurance_fee_existing, fill_value=0)

In [ ]:
total_insurance_fee = aggregate_new_and_existing_loans_insurance_fee(insurance_fee_existing=insurance_fee_existing_loans, insurance_fee_new=insurance_new)
total_insurance_fee

Jan-2023    1,444.49
Feb-2023    3,113.16
Mar-2023    5,020.30
Apr-2023    6,561.55
May-2023    8,051.06
Jun-2023    9,753.30
Jul-2023   10,469.25
Aug-2023   11,565.38
Sep-2023   12,892.42
Oct-2023   15,586.12
Nov-2023   18,849.83
Dec-2023   21,824.44
dtype: float64

In [ ]:

def aggregate_other_income(total_insurance_fee: pd.Series, total_interest_income: pd.Series): 
    return total_insurance_fee.add(total_interest_income, fill_value= 0 )

In [ ]:
other_income = aggregate_other_income(total_insurance_fee=total_insurance_fee, total_interest_income=total_interest_income)
other_income

Jan-2023     1,735.27
Feb-2023    48,317.13
Mar-2023    99,109.29
Apr-2023   151,263.11
May-2023   185,324.00
Jun-2023   212,387.87
Jul-2023   237,399.30
Aug-2023   261,345.31
Sep-2023   300,922.50
Oct-2023   349,853.76
Nov-2023   422,143.20
Dec-2023   497,086.46
dtype: float64

## **Depreciation**


In [ ]:
details_of_existing_assets

,asset,depreciation,net_book_value,remaining_useful_life,cost,useful_life_when_purchased,method
0,Vehicles,0.20,10000,5,20000,10,straight_line
1,Buildings,0.10,50000,20,80000,25,reducing_balance


In [ ]:
def cal_depreciation_and_nbv(
    depreciation_rate: float, remaining_useful_life: int, nbv: float
) -> (np.ndarray, np.ndarray):
    net_book_value = (
        np.power((1 - depreciation_rate),
                 range(1, remaining_useful_life + 1)) * nbv
    )
    depreciation = -np.diff(
        np.power((1 - depreciation_rate),
                 range(0, remaining_useful_life + 1)) * nbv
    )
    return net_book_value, depreciation


def create_depreciation_and_nbv_series_index(
    details_of_assets, remaining_useful_life, asset, new_assets: bool, valuation_date
):
    if new_assets:
        purchase_date = details_of_assets.loc[
            details_of_assets.asset == asset, "purchase_date"
        ].values[0]
        index = pd.period_range(
            purchase_date, periods=remaining_useful_life, freq="M").strftime("%b-%Y")
    else:
        index = pd.period_range(
            valuation_date, periods=remaining_useful_life, freq="M").strftime("%b-%Y")
    return index

In [ ]:
def depreciation_of_non_current_assets(
    details_of_assets: pd.DataFrame,
    valuation_date,
    months_to_forecast,
    new_assets=False,
):
    depreciations = []
    nbvs = []

    for asset in details_of_assets.asset:
        nbv = details_of_assets.loc[
            details_of_assets.asset == asset, "net_book_value"
        ].values[0]
        remaining_useful_life = details_of_assets.loc[
            details_of_assets.asset == asset, "remaining_useful_life"
        ].values[0]
        depreciation_rate = details_of_assets.loc[
            details_of_assets.asset == asset, "depreciation"
        ].values[0]

        index = create_depreciation_and_nbv_series_index(
            details_of_assets=details_of_assets,
            remaining_useful_life=remaining_useful_life,
            asset=asset,
            new_assets=new_assets,
            valuation_date=valuation_date,
        )

        net_book_value, depreciation = cal_depreciation_and_nbv(
            depreciation_rate=depreciation_rate,
            remaining_useful_life=remaining_useful_life,
            nbv=nbv,
        )

        net_book_value = pd.Series(net_book_value, index=index, name=asset)
        depreciation = pd.Series(depreciation, index=index, name=asset)
        depreciations.append(depreciation)
        nbvs.append(net_book_value)

        df_index = pd.period_range(
            valuation_date, periods=months_to_forecast, freq="M").strftime("%b-%Y")

    return {
        "nbvs": pd.concat(nbvs, axis=1).reindex(df_index).fillna(0),
        "depreciations": pd.concat(depreciations, axis=1).reindex(df_index).fillna(0),
    }

In [ ]:
depreciation_of_existing_assets_results = depreciation_of_non_current_assets(
    details_of_assets=details_of_existing_assets,
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)
depreciation_of_new_assets = depreciation_of_non_current_assets(
    details_of_assets=details_of_new_assets,
    valuation_date=VALUATION_DATE,
    months_to_forecast=MONTHS_TO_FORECAST,
    new_assets=True,
)

In [ ]:
nbvs_existing_assets = depreciation_of_existing_assets_results["nbvs"]
nbvs_existing_assets

,Vehicles,Buildings
Jan-2023,"8,000.00","45,000.00"
Feb-2023,"6,400.00","40,500.00"
Mar-2023,"5,120.00","36,450.00"
Apr-2023,"4,096.00","32,805.00"
May-2023,"3,276.80","29,524.50"
Jun-2023,0.00,"26,572.05"
Jul-2023,0.00,"23,914.85"
Aug-2023,0.00,"21,523.36"
Sep-2023,0.00,"19,371.02"
Oct-2023,0.00,"17,433.92"


In [ ]:
depreciation_existing_assets = depreciation_of_existing_assets_results["depreciations"]
depreciation_existing_assets

,Vehicles,Buildings
Jan-2023,"2,000.00","5,000.00"
Feb-2023,"1,600.00","4,500.00"
Mar-2023,"1,280.00","4,050.00"
Apr-2023,"1,024.00","3,645.00"
May-2023,819.20,"3,280.50"
Jun-2023,0.00,"2,952.45"
Jul-2023,0.00,"2,657.20"
Aug-2023,0.00,"2,391.48"
Sep-2023,0.00,"2,152.34"
Oct-2023,0.00,"1,937.10"


In [ ]:
nbvs_new_assets = depreciation_of_new_assets["nbvs"]
nbvs_new_assets

,Vehicle,Building
Jan-2023,"7,500.00","45,000.00"
Feb-2023,"5,625.00","40,500.00"
Mar-2023,"4,218.75","36,450.00"
Apr-2023,"3,164.06","32,805.00"
May-2023,"2,373.05","29,524.50"
Jun-2023,0.00,"26,572.05"
Jul-2023,0.00,"23,914.85"
Aug-2023,0.00,"21,523.36"
Sep-2023,0.00,"19,371.02"
Oct-2023,0.00,"17,433.92"


In [ ]:
depreciation_new_assets = depreciation_of_new_assets["depreciations"]
depreciation_new_assets

,Vehicle,Building
Jan-2023,"2,500.00","5,000.00"
Feb-2023,"1,875.00","4,500.00"
Mar-2023,"1,406.25","4,050.00"
Apr-2023,"1,054.69","3,645.00"
May-2023,791.02,"3,280.50"
Jun-2023,0.00,"2,952.45"
Jul-2023,0.00,"2,657.20"
Aug-2023,0.00,"2,391.48"
Sep-2023,0.00,"2,152.34"
Oct-2023,0.00,"1,937.10"


### **Straight Line**

In [ ]:
details_of_existing_assets

,asset,depreciation,net_book_value,remaining_useful_life,cost,useful_life_when_purchased,method
0,Vehicles,0.20,10000,5,20000,10,straight_line
1,Buildings,0.10,50000,20,80000,25,reducing_balance


In [ ]:
straight_line_depreciation = details_of_existing_assets['depreciation'] *  details_of_existing_assets['cost']
straight_line_depreciation

0   4,000.00
1   8,000.00
dtype: float64

In [ ]:
def calculate_straight_line_depreciation(
    details_of_assets: pd.DataFrame, 
):

    depreciation = details_of_assets['cost'] * details_of_assets['depreciation']
    remaining_useful_life = details_of_assets['remaining_useful_life']
    depreciations = []
    for i in depreciation.index:
        temp = pd.Series(
            np.repeat(
                depreciation.loc[i],
                remaining_useful_life.loc[i],
            ),
            name= details_of_assets['asset'].loc[i],
        )

        depreciations.append(temp)
    depreciations = pd.concat(depreciations, axis=1).T.fillna(0)
    depreciations = depreciations.iloc[:, :MONTHS_TO_FORECAST]
    depreciations.columns = generate_columns(VALUATION_DATE, period=MONTHS_TO_FORECAST)
    depreciations

    return depreciations

In [ ]:
straight_line_depreciation = calculate_straight_line_depreciation(
  details_of_assets=details_of_existing_assets
)
straight_line_depreciation

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Vehicles,"4,000.00","4,000.00","4,000.00","4,000.00","4,000.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00
Buildings,"8,000.00","8,000.00","8,000.00","8,000.00","8,000.00","8,000.00","8,000.00","8,000.00","8,000.00","8,000.00","8,000.00","8,000.00"


In [ ]:
(details_of_existing_assets.set_index('asset').net_book_value - straight_line_depreciation.cumsum(axis = 1).T).T

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Vehicles,"6,000.00","2,000.00","-2,000.00","-6,000.00","-10,000.00","-10,000.00","-10,000.00","-10,000.00","-10,000.00","-10,000.00","-10,000.00","-10,000.00"
Buildings,"42,000.00","34,000.00","26,000.00","18,000.00","10,000.00","2,000.00","-6,000.00","-14,000.00","-22,000.00","-30,000.00","-38,000.00","-46,000.00"


In [ ]:
straight_line_depreciation.cumsum(axis = 1)

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Vehicles,"4,000.00","8,000.00","12,000.00","16,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00","20,000.00"
Buildings,"8,000.00","16,000.00","24,000.00","32,000.00","40,000.00","48,000.00","56,000.00","64,000.00","72,000.00","80,000.00","88,000.00","96,000.00"


In [ ]:
details_of_existing_assets.set_index('asset')['cost'].reindex_like(straight_line_depreciation)

Vehicles     20000
Buildings    80000
Name: cost, dtype: int64

In [ ]:
details_of_existing_assets.set_index('asset')['cost']

asset
Vehicles     20000
Buildings    80000
Name: cost, dtype: int64

In [ ]:
details_of_new_assets

,asset,depreciation,net_book_value,remaining_useful_life,method,purchase_date,cost
0,Vehicle,0.25,10000,5,reducing_balance,01/06/2023,10000
1,Building,0.10,50000,20,reducing_balance,01/09/2023,50000


In [ ]:
reducing_balance_new_assets = details_of_new_assets.loc[details_of_new_assets.method == 'reducing_balance']
reducing_balance_new_assets

,asset,depreciation,net_book_value,remaining_useful_life,method,purchase_date,cost
0,Vehicle,0.25,10000,5,reducing_balance,01/06/2023,10000
1,Building,0.10,50000,20,reducing_balance,01/09/2023,50000


In [ ]:
straight_line_new_assets = details_of_new_assets.loc[details_of_new_assets.method != 'reducing_balance']
straight_line_new_assets

,asset,depreciation,net_book_value,remaining_useful_life,method,purchase_date,cost


In [ ]:
straight_line_existing_assets = details_of_existing_assets.loc[details_of_existing_assets.method != 'reducing_balance']
straight_line_existing_assets

,asset,depreciation,net_book_value,remaining_useful_life,cost,useful_life_when_purchased,method
0,Vehicles,0.20,10000,5,20000,10,straight_line


In [ ]:
reducing_balance_existing_assets = details_of_existing_assets.loc[details_of_existing_assets.method == 'reducing_balance']
reducing_balance_existing_assets

,asset,depreciation,net_book_value,remaining_useful_life,cost,useful_life_when_purchased,method
1,Buildings,0.10,50000,20,80000,25,reducing_balance


## **Salaries**

### **Agent Salaries**

In [ ]:
agent_salaries = parameters.loc['AGENT_SALARY'] * parameters.loc['CONSUMER_PVT_NUMBER_OF_AGENTS'] + parameters.loc['AGENT_COMMISSION'] * (consumer_pvt_disbursements + consumer_ssb_disbursements)
agent_salaries

2023-01    6,000.00
2023-02    7,500.00
2023-03    9,900.00
2023-04   11,000.00
2023-05   13,200.00
2023-06   14,100.00
2023-07   16,450.00
2023-08   17,625.00
2023-09   18,750.00
2023-10   18,750.00
2023-11   18,750.00
2023-12   18,750.00
Freq: M, dtype: float64

In [ ]:
def calculate_agent_salaries(consumer_pvt_disbursements: pd.Series, consumer_ssb_disbursements: pd.Series, parameters: pd.DataFrame): 
    agent_salaries = parameters.loc['AGENT_SALARY'] * parameters.loc['CONSUMER_PVT_NUMBER_OF_AGENTS'] + parameters.loc['AGENT_COMMISSION'] * (consumer_pvt_disbursements + consumer_ssb_disbursements)
    return agent_salaries


In [ ]:
agent_salaries = calculate_agent_salaries(
    consumer_pvt_disbursements=consumer_pvt_disbursements,
    consumer_ssb_disbursements=consumer_ssb_disbursements,
    parameters=parameters,
)
agent_salaries

2023-01    6,000.00
2023-02    7,500.00
2023-03    9,900.00
2023-04   11,000.00
2023-05   13,200.00
2023-06   14,100.00
2023-07   16,450.00
2023-08   17,625.00
2023-09   18,750.00
2023-10   18,750.00
2023-11   18,750.00
2023-12   18,750.00
Freq: M, dtype: float64

### **Loan Officer Salaries**

In [ ]:
credit_officer_salaries = parameters.loc['CREDIT_OFFICER_SALARY'] * parameters.loc['SME_NUMBER_OF_CREDIT_OFFICERS'] + parameters.loc['CREDIT_OFFICER_COMMISSION'] * sme_disbursements
credit_officer_salaries 

2023-01    4,200.00
2023-02    4,920.00
2023-03    5,400.00
2023-04    4,200.00
2023-05    4,140.00
2023-06    4,680.00
2023-07    5,400.00
2023-08    7,350.00
2023-09    8,400.00
2023-10   11,700.00
2023-11   13,050.00
2023-12   13,050.00
Freq: M, dtype: float64

In [ ]:

def calculate_credit_officer_salaries(sme_disbursements: pd.Series, parameters: pd.DataFrame): 
    credit_officer_salaries = parameters.loc['CREDIT_OFFICER_SALARY'] * parameters.loc['SME_NUMBER_OF_CREDIT_OFFICERS'] + parameters.loc['CREDIT_OFFICER_COMMISSION'] * sme_disbursements
    return credit_officer_salaries


In [ ]:
credit_officer_salaries = calculate_credit_officer_salaries(sme_disbursements= sme_disbursements, parameters=parameters)
credit_officer_salaries

2023-01    4,200.00
2023-02    4,920.00
2023-03    5,400.00
2023-04    4,200.00
2023-05    4,140.00
2023-06    4,680.00
2023-07    5,400.00
2023-08    7,350.00
2023-09    8,400.00
2023-10   11,700.00
2023-11   13,050.00
2023-12   13,050.00
Freq: M, dtype: float64

In [ ]:
total_salaries = agent_salaries + credit_officer_salaries + parameters.loc['OTHER_STAFF_SALARY']
total_salaries

2023-01   27,200.00
2023-02   29,420.00
2023-03   32,300.00
2023-04   32,200.00
2023-05   34,340.00
2023-06   35,780.00
2023-07   38,850.00
2023-08   41,975.00
2023-09   44,150.00
2023-10   47,450.00
2023-11   48,800.00
2023-12   48,800.00
Freq: M, dtype: float64

In [ ]:
def calculate_total_salaries(agent_salaries, credit_officer_salaries, other_staff_salary): 
    total_salaries = agent_salaries + credit_officer_salaries + other_staff_salary
    total_salaries.index = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
    return total_salaries


In [ ]:
total_salaries = calculate_total_salaries(agent_salaries=agent_salaries, credit_officer_salaries=credit_officer_salaries, other_staff_salary=parameters.loc['OTHER_STAFF_SALARY'])
total_salaries

Jan-2023   27,200.00
Feb-2023   29,420.00
Mar-2023   32,300.00
Apr-2023   32,200.00
May-2023   34,340.00
Jun-2023   35,780.00
Jul-2023   38,850.00
Aug-2023   41,975.00
Sep-2023   44,150.00
Oct-2023   47,450.00
Nov-2023   48,800.00
Dec-2023   48,800.00
dtype: float64

## **Trade Receivables**

In [ ]:
opening_balances

,value
balance,
TRADE_RECEIVABLES,1000000
TRADE_PAYABLES,940000
LOAN_BOOK,2800000
CASH_ON_HAND,331000
ISSUED_SHARE_CAPITAL,7000
SHARE_PREMIUM,110
OTHER_COMPONENTS_OF_EQUITY,0
TREASURY_SHARES,0
RETAINED_EARNINGS,200000


In [ ]:
opening_balances.loc['TRADE_RECEIVABLES']

value    1000000
Name: TRADE_RECEIVABLES, dtype: int64

In [ ]:
receipts_from_trade_receivables = parameters.loc["RECEIPTS_FROM_TRADE_RECEIVABLES"]
receipts_from_trade_receivables.index = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
receipts_from_trade_receivables

Jan-2023   1,000.00
Feb-2023   1,000.00
Mar-2023   1,000.00
Apr-2023   1,000.00
May-2023   1,000.00
Jun-2023   1,000.00
Jul-2023   1,000.00
Aug-2023   1,000.00
Sep-2023   1,000.00
Oct-2023   1,000.00
Nov-2023   1,000.00
Dec-2023   1,000.00
Name: RECEIPTS_FROM_TRADE_RECEIVABLES, dtype: float64

In [ ]:
new_trade_receivables = parameters.loc['NEW_TRADE_RECEIVABLES']
new_trade_receivables.index = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
new_trade_receivables

Jan-2023   500.00
Feb-2023   500.00
Mar-2023   500.00
Apr-2023   500.00
May-2023   500.00
Jun-2023   500.00
Jul-2023   500.00
Aug-2023   500.00
Sep-2023   500.00
Oct-2023   500.00
Nov-2023   500.00
Dec-2023   500.00
Name: NEW_TRADE_RECEIVABLES, dtype: float64

In [ ]:
trade_receivables = pd.DataFrame(
    columns=generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST),
    index=["opening_balance", "new_trade_receivables", 'receipts_from_trade_receivables', 'closing_balance'],data = 0
)
trade_receivables

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
opening_balance,0,0,0,0,0,0,0,0,0,0,0,0
new_trade_receivables,0,0,0,0,0,0,0,0,0,0,0,0
receipts_from_trade_receivables,0,0,0,0,0,0,0,0,0,0,0,0
closing_balance,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
trade_receivables.loc['new_trade_receivables'] = new_trade_receivables
trade_receivables.loc['receipts_from_trade_receivables'] = -receipts_from_trade_receivables
trade_receivables.iloc[0,0] = opening_balances.loc['TRADE_RECEIVABLES']
trade_receivables.columns = pd.PeriodIndex(trade_receivables.columns, freq="M")
trade_receivables

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
opening_balance,1000000,0,0,0,0,0,0,0,0,0,0,0
new_trade_receivables,500,500,500,500,500,500,500,500,500,500,500,500
receipts_from_trade_receivables,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000
closing_balance,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
for period in trade_receivables.columns:
    trade_receivables.loc["closing_balance", period] = trade_receivables.loc[
        "opening_balance":"receipts_from_trade_receivables", period
    ].sum()
    if period == trade_receivables.columns[-1]: 
        break
    trade_receivables.loc['opening_balance',period + 1] = trade_receivables.loc['closing_balance',period]
# trade_receivables.columns = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
trade_receivables

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
opening_balance,1000000,999500,999000,998500,998000,997500,997000,996500,996000,995500,995000,994500
new_trade_receivables,500,500,500,500,500,500,500,500,500,500,500,500
receipts_from_trade_receivables,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000
closing_balance,999500,999000,998500,998000,997500,997000,996500,996000,995500,995000,994500,994000


In [ ]:
def generate_trade_receivables_schedule(opening_trade_receivables, receipts_from_trade_receivables, new_trade_receivables): 
    receipts_from_trade_receivables.index = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
    new_trade_receivables.index = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)

    trade_receivables = pd.DataFrame(
    columns=generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST),
    index=["opening_balance", "new_trade_receivables", 'receipts_from_trade_receivables', 'closing_balance'],data = 0
    )
    
    trade_receivables.loc['new_trade_receivables'] = new_trade_receivables
    trade_receivables.loc['receipts_from_trade_receivables'] = -receipts_from_trade_receivables
    trade_receivables.iloc[0,0] = opening_trade_receivables
    trade_receivables.columns = pd.PeriodIndex(trade_receivables.columns, freq="M")


    for period in trade_receivables.columns:
        trade_receivables.loc["closing_balance", period] = trade_receivables.loc[
            "opening_balance":"receipts_from_trade_receivables", period
        ].sum()
        if period == trade_receivables.columns[-1]: 
            break
        trade_receivables.loc['opening_balance',period + 1] = trade_receivables.loc['closing_balance',period]
    return trade_receivables    

In [ ]:
generate_trade_receivables_schedule(opening_trade_receivables=opening_balances.loc['TRADE_RECEIVABLES'], 
                                    receipts_from_trade_receivables=parameters.loc['RECEIPTS_FROM_TRADE_RECEIVABLES'], 
                                    new_trade_receivables=parameters.loc['NEW_TRADE_RECEIVABLES'])

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
opening_balance,1000000,999500,999000,998500,998000,997500,997000,996500,996000,995500,995000,994500
new_trade_receivables,500,500,500,500,500,500,500,500,500,500,500,500
receipts_from_trade_receivables,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000,-1000
closing_balance,999500,999000,998500,998000,997500,997000,996500,996000,995500,995000,994500,994000


In [ ]:
provision_for_bad_debts = trade_receivables.loc['closing_balance'] * parameters.loc['TRADE_RECEIVABLES_PROVISION_FOR_BAD_DEBTS']
provision_for_bad_debts.index = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
provision_for_bad_debts

Jan-2023   99.95
Feb-2023   99.90
Mar-2023   99.85
Apr-2023   99.80
May-2023   99.75
Jun-2023   99.70
Jul-2023   99.65
Aug-2023   99.60
Sep-2023   99.55
Oct-2023   99.50
Nov-2023   99.45
Dec-2023   99.40
dtype: float64

In [ ]:
def calculate_provision_for_bad_debts(trade_receivables,trade_receivables_provision_for_bad_debts):
    provision_for_bad_debts = trade_receivables * trade_receivables_provision_for_bad_debts
    provision_for_bad_debts.index = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
    return  provision_for_bad_debts


## **Income Statement**


### **Forecast Variable Expenses**


In [ ]:
def calculate_variable_expenses(
    variable_inputs_income_statement: pd.DataFrame, parameters: pd.DataFrame
):
    inflation_rates = (parameters.loc["INFLATION_RATE"] + 1).cumprod()
    inflation_rates.index = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
    mean_expenses = pd.DataFrame(
        np.repeat(variable_inputs_income_statement.mean(), 12).values.reshape(
            variable_inputs_income_statement.T.index.shape[0], MONTHS_TO_FORECAST
        ),
        index=variable_inputs_income_statement.T.index,
    )
    mean_expenses.columns = generate_columns(VALUATION_DATE, period=MONTHS_TO_FORECAST)
    return mean_expenses * inflation_rates

In [ ]:
variable_expenses = calculate_variable_expenses(variable_inputs_income_statement, parameters)
variable_expenses

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Travel Costs,"7,080.16","7,221.76","7,366.20","7,513.52","7,663.79","7,817.07","7,973.41","8,132.88","8,295.54","8,461.45","8,630.68","8,803.29"
Entertainment,"7,760.67","7,915.88","8,074.20","8,235.69","8,400.40","8,568.41","8,739.77","8,914.57","9,092.86","9,274.72","9,460.21","9,649.42"
Marketing Costs,"7,796.37","7,952.30","8,111.34","8,273.57","8,439.04","8,607.82","8,779.98","8,955.58","9,134.69","9,317.38","9,503.73","9,693.81"
Group Marketing Costs,"7,659.18","7,812.36","7,968.61","8,127.98","8,290.54","8,456.35","8,625.48","8,797.99","8,973.95","9,153.43","9,336.50","9,523.23"
Rental Costs,"7,749.79","7,904.79","8,062.88","8,224.14","8,388.62","8,556.39","8,727.52","8,902.07","9,080.11","9,261.72","9,446.95","9,635.89"
Subscriptions,"8,444.75","8,613.64","8,785.92","8,961.64","9,140.87","9,323.69","9,510.16","9,700.36","9,894.37","10,092.26","10,294.10","10,499.99"
Insurance,"7,590.16","7,741.96","7,896.80","8,054.74","8,215.83","8,380.15","8,547.75","8,718.71","8,893.08","9,070.94","9,252.36","9,437.41"
Repairs And Maintenance,"8,262.68","8,427.93","8,596.49","8,768.42","8,943.79","9,122.67","9,305.12","9,491.22","9,681.05","9,874.67","10,072.16","10,273.60"
Utilities,"7,537.97","7,688.73","7,842.50","7,999.35","8,159.34","8,322.53","8,488.98","8,658.76","8,831.93","9,008.57","9,188.74","9,372.52"


In [ ]:
def calculate_provision_for_credit_loss(disbursements, provision_for_credit_loss):
    return disbursements * provision_for_credit_loss

In [ ]:
sme_disbursements * parameters.loc['SME_PROVISION_FOR_CREDIT_LOSS']

2023-01    7,200.00
2023-02    8,640.00
2023-03    9,600.00
2023-04    7,200.00
2023-05    6,480.00
2023-06    7,560.00
2023-07    8,400.00
2023-08   11,900.00
2023-09   14,000.00
2023-10   19,800.00
2023-11   22,500.00
2023-12   22,500.00
Freq: M, dtype: float64

In [ ]:
sme_provision_for_credit_loss = calculate_provision_for_credit_loss(
    sme_disbursements, parameters.loc['SME_PROVISION_FOR_CREDIT_LOSS'])
b2b_provision_for_credit_loss = calculate_provision_for_credit_loss(
    b2b_disbursements, parameters.loc['B2B_PROVISION_FOR_CREDIT_LOSS'])
consumer_ssb_provision_for_credit_loss = calculate_provision_for_credit_loss(
    consumer_ssb_disbursements, parameters.loc['CONSUMER_SSB_PROVISION_FOR_CREDIT_LOSS'])
consumer_pvt_provision_for_credit_loss = calculate_provision_for_credit_loss(
    consumer_pvt_disbursements, parameters.loc['CONSUMER_PVT_PROVISION_FOR_CREDIT_LOSS'])

In [ ]:
sme_provision_for_credit_loss

2023-01    7,200.00
2023-02    8,640.00
2023-03    9,600.00
2023-04    7,200.00
2023-05    6,480.00
2023-06    7,560.00
2023-07    8,400.00
2023-08   11,900.00
2023-09   14,000.00
2023-10   19,800.00
2023-11   22,500.00
2023-12   22,500.00
Freq: M, dtype: float64

In [ ]:
def add_series(list_of_series:list): 
    return pd.concat(list_of_series, axis = 1).sum(axis =1)

In [ ]:
income_statement_index = pd.read_csv("./data_templates/income_statement_index.csv")
income_statement_index['INCOME STATEMENT'] = income_statement_index['INCOME STATEMENT'].str.strip()
income_statement_index

,INCOME STATEMENT
0,Interest Income
1,Other Income
2,Total Revenue
3,MANAGEMENT EXPENSES
4,STAFF COSTS
5,Salaries
6,Pensions & Statutory Contributions
7,Training
8,Bonus Provision
9,Retrenchments


In [ ]:
income_statement_template = pd.DataFrame(columns=generate_columns(
    VALUATION_DATE, MONTHS_TO_FORECAST), index=income_statement_index['INCOME STATEMENT'])
income_statement_template

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MANAGEMENT EXPENSES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
STAFF COSTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def remove_na_from_headings(income_statement_template: pd.DataFrame):
    headings = income_statement_template.index[income_statement_template.index.str.isupper()]
    income_statement_template.loc[headings] = ""
    return income_statement_template

In [ ]:
income_statement_template = remove_na_from_headings(income_statement_template)
income_statement_template

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **Interest Income Aggregation**


In [ ]:
existing_loans_schedules['interest'].sum()

Jan-2023   290.78
Feb-2023   283.97
Mar-2023   277.09
Apr-2023   270.13
May-2023   263.09
Jun-2023   255.98
Jul-2023   248.78
Aug-2023   241.51
Sep-2023   234.16
Oct-2023   226.73
Nov-2023   219.21
Dec-2023     0.00
dtype: float64

In [ ]:
interest_income = add_series(
    [
        sme_loan_schedules["interest"],
        consumer_pvt_loan_schedules["interest"],
        consumer_ssb_loan_schedules["interest"],
        b2b_loan_schedules["interest"],
    ]
)
interest_income.index = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
total_interest_income = add_series([interest_income,existing_loans_schedules['interest'].sum()])
total_interest_income

Jan-2023   189,400.85
Feb-2023   228,667.56
Mar-2023   268,375.14
Apr-2023   257,174.38
May-2023   258,655.84
Jun-2023   275,190.55
Jul-2023   295,104.59
Aug-2023   336,792.80
Sep-2023   329,521.29
Oct-2023   336,211.61
Nov-2023   262,711.85
Dec-2023   138,925.00
dtype: float64

In [ ]:
admin_fee = add_series([sme_admin_fee, consumer_ssb_admin_fee, b2b_admin_fee, consumer_pvt_admin_fee, admin_fee_existing_loans])
admin_fee

Jan-2023    3,533.56
Feb-2023    7,715.24
Mar-2023   12,493.08
Apr-2023   16,356.22
May-2023   20,090.00
Jun-2023   24,355.59
Jul-2023   26,155.45
Aug-2023   28,905.78
Sep-2023   32,233.38
Oct-2023   38,977.64
Nov-2023   47,136.91
Dec-2023   54,573.45
dtype: float64

In [ ]:
insurance_fee = add_series([sme_insurance, consumer_ssb_insurance, b2b_insurance, consumer_pvt_insurance, insurance_fee_existing_loans])
insurance_fee

Jan-2023    1,444.49
Feb-2023    3,113.16
Mar-2023    5,020.30
Apr-2023    6,561.55
May-2023    8,051.06
Jun-2023    9,753.30
Jul-2023   10,469.25
Aug-2023   11,565.38
Sep-2023   12,892.42
Oct-2023   15,586.12
Nov-2023   18,849.83
Dec-2023   21,824.44
dtype: float64

In [ ]:
other_income = add_series([insurance_fee, admin_fee])
other_income

Jan-2023    4,978.06
Feb-2023   10,828.41
Mar-2023   17,513.37
Apr-2023   22,917.78
May-2023   28,141.06
Jun-2023   34,108.89
Jul-2023   36,624.70
Aug-2023   40,471.16
Sep-2023   45,125.79
Oct-2023   54,563.77
Nov-2023   65,986.74
Dec-2023   76,397.89
dtype: float64

In [ ]:
income_statement_template

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Revenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def insert_revenue(income_statement, interest_income, admin_fee):
    income_statement.loc["Interest Income"] = total_interest_income
    
    income_statement.loc["Other Income"] = other_income
    income_statement.loc["Total Revenue"] = (
        income_statement.loc["Interest Income"] +
        income_statement.loc["Other Income"]
    )
    return income_statement

In [ ]:
income_statement = insert_revenue(
    income_statement=income_statement_template,
    interest_income=interest_income,
    admin_fee=admin_fee,
)
income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bonus Provision,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def insert_static_and_variable_inputs(
    income_statement, static_inputs_income_statement, variable_expenses
):
    income_statement.loc[
        static_inputs_income_statement.T.index
    ] = static_inputs_income_statement.T
    income_statement.loc[variable_expenses.index] = variable_expenses
    return income_statement

In [ ]:
income_statement = insert_static_and_variable_inputs(
    income_statement=income_statement,
    static_inputs_income_statement=static_inputs_income_statement, variable_expenses=variable_expenses
)
income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pensions & Statutory Contributions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [ ]:
total_salaries

Jan-2023   27,200.00
Feb-2023   29,420.00
Mar-2023   32,300.00
Apr-2023   32,200.00
May-2023   34,340.00
Jun-2023   35,780.00
Jul-2023   38,850.00
Aug-2023   41,975.00
Sep-2023   44,150.00
Oct-2023   47,450.00
Nov-2023   48,800.00
Dec-2023   48,800.00
dtype: float64

In [ ]:
def calculate_pensions_and_statutory_contributions(
    salaries: pd.Series,
    pensions_and_statutory_contributions_percentage: pd.Series,
):
    pensions_and_statutory_contributions_percentage.index = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
    pensions_and_statutory_contributions = (
        salaries
        * pensions_and_statutory_contributions_percentage
    )
    return pensions_and_statutory_contributions

In [ ]:
def insert_salaries_and_pensions_and_statutory_contributions(
    income_statement: pd.DataFrame, pensions_and_statutory_contributions: pd.Series, salaries: pd.Series
):
    income_statement.loc[
        "Pensions & Statutory Contributions"
    ] = pensions_and_statutory_contributions
    income_statement.loc['Salaries'] = salaries
    return income_statement

In [ ]:
pensions_and_statutory_contributions = calculate_pensions_and_statutory_contributions(
    salaries=total_salaries, 
    pensions_and_statutory_contributions_percentage=parameters.loc['PENSION_AND_STATUROTY_CONTRIBUTIONS_PERCENT'],
)
pensions_and_statutory_contributions

Jan-2023   3,089.92
Feb-2023   3,342.11
Mar-2023   3,669.28
Apr-2023   3,657.92
May-2023   3,901.02
Jun-2023   4,064.61
Jul-2023   4,413.36
Aug-2023   4,768.36
Sep-2023   5,015.44
Oct-2023   5,390.32
Nov-2023   5,543.68
Dec-2023   5,543.68
dtype: float64

In [ ]:
income_statement = insert_salaries_and_pensions_and_statutory_contributions(
    income_statement=income_statement,
    pensions_and_statutory_contributions=pensions_and_statutory_contributions,
    salaries=total_salaries,
)
income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [ ]:
def aggregate_depreciation(depreciation_of_existing_asset: dict, depreciation_of_new_assets: dict):
    depreciation = depreciation_of_existing_asset['depreciations'].sum(
        axis=1) + depreciation_of_new_assets['depreciations'].sum(axis=1)
    return depreciation

In [ ]:
depreciation = aggregate_depreciation(
    depreciation_of_existing_asset=depreciation_of_existing_assets_results,
    depreciation_of_new_assets=depreciation_of_new_assets,
)
depreciation

Jan-2023   14,500.00
Feb-2023   12,475.00
Mar-2023   10,786.25
Apr-2023    9,368.69
May-2023    8,171.22
Jun-2023    5,904.90
Jul-2023    5,314.41
Aug-2023    4,782.97
Sep-2023    4,304.67
Oct-2023    3,874.20
Nov-2023    3,486.78
Dec-2023    3,138.11
dtype: float64

In [ ]:
def insert_depreciation(income_statement: pd.DataFrame, depreciation: pd.Series):
    income_statement.loc["Depreciation"] = depreciation
    return income_statement

In [ ]:
income_statement = insert_depreciation(
    income_statement=income_statement, depreciation=depreciation)
income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [ ]:
provision_for_bad_debts

Jan-2023   99.95
Feb-2023   99.90
Mar-2023   99.85
Apr-2023   99.80
May-2023   99.75
Jun-2023   99.70
Jul-2023   99.65
Aug-2023   99.60
Sep-2023   99.55
Oct-2023   99.50
Nov-2023   99.45
Dec-2023   99.40
dtype: float64

In [ ]:
provision_for_credit_loss = add_series(
    [
        b2b_provision_for_credit_loss,
        sme_provision_for_credit_loss,
        consumer_pvt_provision_for_credit_loss,
        consumer_ssb_provision_for_credit_loss,
    ]
)
provision_for_credit_loss

2023-01   10,060.00
2023-02   12,190.00
2023-03   14,380.00
2023-04   12,500.00
2023-05   12,820.00
2023-06   14,440.00
2023-07   16,410.00
2023-08   20,475.00
2023-09   23,250.00
2023-10   29,050.00
2023-11   31,750.00
2023-12   31,750.00
Freq: M, dtype: float64

In [ ]:
def calculate_change_in_provision_for_credit_loss(provision_for_credit_loss: pd.Series, opening_balances: pd.Series): 
    provision_for_credit_loss.loc[provision_for_credit_loss.index[0]-1] = float(opening_balances.loc['PROVISION_FOR_CREDIT_LOSS'].iat[0])
    provision_for_credit_loss = provision_for_credit_loss.sort_index().diff().dropna()
    provision_for_credit_loss.index = generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
    return provision_for_credit_loss

In [ ]:
change_in_provisin_for_credit_loss = calculate_change_in_provision_for_credit_loss(provision_for_credit_loss, opening_balances)
change_in_provisin_for_credit_loss

Jan-2023       60.00
Feb-2023    2,130.00
Mar-2023    2,190.00
Apr-2023   -1,880.00
May-2023      320.00
Jun-2023    1,620.00
Jul-2023    1,970.00
Aug-2023    4,065.00
Sep-2023    2,775.00
Oct-2023    5,800.00
Nov-2023    2,700.00
Dec-2023        0.00
dtype: float64

In [ ]:
def insert_bad_debts_provision(income_statement, provision_for_bad_debts, change_in_provisin_for_credit_loss):
    income_statement.loc["Bad Debts Provision"] = add_series([change_in_provisin_for_credit_loss,provision_for_bad_debts])
    return income_statement

In [ ]:
income_statement = insert_bad_debts_provision(
    income_statement=income_statement, provision_for_bad_debts=provision_for_bad_debts, change_in_provisin_for_credit_loss= change_in_provisin_for_credit_loss
)

income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [ ]:
def calculate_business_acqusition(
    business_acquisition_percent: float,
    agent_contribution_percent: float,
    consumer_ssb_disbursements: pd.Series,
    consumer_pvt_disbursements: pd.Series,
):
    consumer_ssb = business_acquisition_percent * consumer_ssb_disbursements
    agent_contribution = agent_contribution_percent * consumer_pvt_disbursements
    acquisition = agent_contribution * business_acquisition_percent
    total_acquisition = consumer_ssb + acquisition
    total_acquisition.index = total_acquisition.index.strftime("%b-%Y")
    return total_acquisition

In [ ]:
business_acquisition = calculate_business_acqusition(
    business_acquisition_percent=parameters.loc['BUSINESS_ACQUISITION_PERCENT'],
    agent_contribution_percent=parameters.loc['AGENT_CONTRIBUTION_PERCENT'],
    consumer_ssb_disbursements=consumer_ssb_disbursements,
    consumer_pvt_disbursements=consumer_pvt_disbursements,
)
business_acquisition

Jan-2023   1,620.00
Feb-2023   2,025.00
Mar-2023   2,484.00
Apr-2023   2,760.00
May-2023   3,312.00
Jun-2023   3,366.00
Jul-2023   3,927.00
Aug-2023   4,207.50
Sep-2023   4,275.00
Oct-2023   4,275.00
Nov-2023   4,275.00
Dec-2023   4,275.00
dtype: float64

In [ ]:
def insert_business_acquisition(
    income_statement: pd.DataFrame, business_acquisition: pd.Series
):
    income_statement.loc["Business Acquisition"] = business_acquisition
    return income_statement

In [ ]:
income_statement = insert_business_acquisition(
    income_statement=income_statement, business_acquisition=business_acquisition)
income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [ ]:
def aggregate_staff_costs(income_statement):
    income_statement.loc["Total Staff Costs"] = income_statement.loc[
        "Salaries":"CILL"
    ].fillna(0).sum()
    return income_statement


def aggregate_travel_and_entertainment(income_statement):
    income_statement.loc["Total Travel & Entertainment"] = income_statement.loc[
        "Travel Costs":"Entertainment"
    ].fillna(0).sum()
    return income_statement


def aggregate_marketing_and_public_relations(income_statement):
    income_statement.loc["Total Marketing And Public Relations"] = income_statement.loc[
        "Marketing Costs":"Donations"
    ].fillna(0).sum()
    return income_statement


def aggregate_office_costs(income_statement):
    income_statement.loc["Total Office Costs"] = income_statement.loc[
        "Rental Costs":"Fines And Penalties"
    ].fillna(0).sum()
    return income_statement


def aggregate_professional_fees(income_statement):
    income_statement.loc["Total Professional Fees"] = income_statement.loc[
        "Auditors Remuneration":"Consultancy Fees"
    ].fillna(0).sum()
    return income_statement


def aggregate_communication_costs(income_statement):
    income_statement.loc["Total Communication Costs"] = income_statement.loc[
        "Telephones":"Courier"
    ].fillna(0).sum()
    return income_statement


def aggregate_motor_vehicle_costs(income_statement):
    income_statement.loc["Total Motor Vehicle Costs"] = income_statement.loc[
        "Fuel":"Motor Vehicle Maintenance Costs"
    ].fillna(0).sum()
    return income_statement


def aggregate_other_costs(income_statement):
    income_statement.loc["Total Other Costs"] = income_statement.loc[
        "Depreciation":"Business Acquisition"
    ].fillna(0).sum()
    return income_statement


def aggregate_investment_income(income_statement):
    income_statement.loc["Total Investment Income"] = income_statement.loc[
        "Rental Income":"Admin Fees"
    ].fillna(0).sum()
    return income_statement


def aggregate_finance_costs(income_statement):
    income_statement.loc["Total Finance Costs"] = income_statement.loc[
        "Finance Costs":"Third Party"
    ].fillna(0).sum()
    return income_statement


def calculate_total_expenses(income_statement):
    income_statement.loc["TOTAL EXPENSES"] = (
        income_statement.loc["Total Staff Costs"]
        + income_statement.loc["Total Travel & Entertainment"]
        + income_statement.loc["Total Marketing And Public Relations"]
        + income_statement.loc["Total Office Costs"]
        + income_statement.loc["Total Professional Fees"]
        + income_statement.loc["Total Communication Costs"]
        + income_statement.loc["Total Motor Vehicle Costs"]
        + income_statement.loc["Total Other Costs"]
    )
    return income_statement


def calculate_ebidta(income_statement):
    income_statement.loc["EBIDTA"] = (
        income_statement.loc["Total Revenue"] -
        income_statement.loc["TOTAL EXPENSES"]
    )
    return income_statement


def calculate_profit_before_tax(income_statement):
    income_statement.loc["PROFIT / (LOSS) BEFORE TAX"] = (
        income_statement.loc["EBIDTA"]
        + income_statement.loc["Total Investment Income"]
        - income_statement.loc["Total Finance Costs"]
    )
    return income_statement


def calculate_tax(income_statement, tax_rate):
    income_statement.loc["Taxation"] = np.maximum(
        income_statement.loc["PROFIT / (LOSS) BEFORE TAX"] * tax_rate, 0)
    return income_statement

In [ ]:
income_statement = aggregate_staff_costs(income_statement)
income_statement = aggregate_travel_and_entertainment(income_statement)
income_statement = aggregate_marketing_and_public_relations(income_statement)
income_statement = aggregate_office_costs(income_statement)
income_statement = aggregate_professional_fees(income_statement)
income_statement = aggregate_communication_costs(income_statement)
income_statement = aggregate_motor_vehicle_costs(income_statement)
income_statement = aggregate_other_costs(income_statement)
income_statement = aggregate_investment_income(income_statement)
income_statement = aggregate_finance_costs(income_statement)

In [ ]:
income_statement = calculate_total_expenses(income_statement)
income_statement = calculate_ebidta(income_statement)

In [ ]:
income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [ ]:
details_of_existing_short_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Life,50000,0.14,10,12,straight_line
1,Zimnat Life,30000,0.13,12,12,straight_line
2,Grandre,100000,0.12,7,12,straight_line
3,FCB,120383,0.18,5,12,straight_line
4,Nurture,500000,0.25,8,12,straight_line
5,NMB,450000,0.22,11,12,straight_line
6,Beramasamba,30000,0.13,3,12,straight_line
7,Dandemutande,25000,0.12,7,12,reducing_balance


In [ ]:
details_of_new_short_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,12,12,01/07/2023,straight_line
1,ZAM,400000,0.25,12,12,01/07/2023,reducing_balance
2,NBS,500000,0.18,12,12,01/09/2023,straight_line
3,ZB,500000,0.21,12,12,01/10/2023,straight_line


In [ ]:
details_of_new_long_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,36,36,01/08/2023,straight_line
1,ZAM,400000,0.25,60,60,01/09/2023,straight_line
2,NBS,500000,0.18,24,24,01/10/2023,straight_line
3,ZB,500000,0.21,18,18,01/10/2023,straight_line


In [ ]:
details_of_existing_long_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Lion Insurance,92800,0.06,60,60,straight_line
1,Zimnat Lion Insurance,50000,0.06,12,12,straight_line
2,Grandre,160000,0.06,36,36,straight_line
3,Minerva,15000,0.08,36,36,straight_line
4,Masawara,5613,0.06,60,60,straight_line
5,Zam-Cargo,364000,0.05,36,36,straight_line


In [ ]:
import pandas as pd
def calculate_interest_expense(df, to_pad, idx):
    remaining_loan_term = df.loc[idx, "remaining_loan_term"]
    loan_interest = df.loc[idx, "interest"]
    interest = np.pad(
        np.repeat(loan_interest, remaining_loan_term), (0, to_pad - remaining_loan_term)
    )
    return interest

def convert_interest_array_to_series(interest_array, start_date):
    return pd.Series(interest_array, index=generate_columns(start_date, len(interest_array)))

def reindex_columns(df, valuation_date, months_to_forecast):
    return df.reindex(generate_columns(valuation_date, months_to_forecast), axis = 1, fill_value = 0)

def determine_start_date(df, valuation_date, idx):
    if 'loan_start_date' in df.columns:
        start_date = df.loc[idx, 'loan_start_date']
    else:
        start_date = valuation_date
    return start_date

In [ ]:
def calculate_interest_expense_on_borrowing(details_of_borrowing, months_to_forecast, valuation_date):
    interests = []
    temp = details_of_borrowing.copy()
    temp = temp.assign(interest=(
        details_of_borrowing['interest_rate'] * details_of_borrowing['principal']) / temp['loan_term'])
    to_pad = temp.loan_term.max()
    for i in range(len(temp)):
        start_date = determine_start_date(temp, valuation_date=valuation_date, idx=i)
        interest = calculate_interest_expense(temp, to_pad=to_pad, idx=i)
        interest = convert_interest_array_to_series(interest_array=interest,start_date=start_date)
        interests.append(interest)
    interest_expense = pd.concat(interests, axis=1).T.fillna(0)
    interest_expense = reindex_columns(interest_expense, valuation_date=valuation_date,months_to_forecast=months_to_forecast)
    interest_expense.index = temp['company']    
    return interest_expense

In [ ]:
interest_expense_existing_long_term_borrowing = calculate_interest_expense_on_borrowing(
    details_of_borrowing=details_of_existing_long_term_borrowing,
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,
)
interest_expense_existing_short_term_borrowing = calculate_interest_expense_on_borrowing(
    details_of_borrowing=details_of_existing_short_term_borrowing,
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,)
interest_expense_new_short_term_borrowing = calculate_interest_expense_on_borrowing(
    details_of_borrowing=details_of_new_short_term_borrowing,
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,)    
interest_expense_new_long_term_borrowing = calculate_interest_expense_on_borrowing(
    details_of_borrowing=details_of_new_long_term_borrowing,
    months_to_forecast=MONTHS_TO_FORECAST,
    valuation_date=VALUATION_DATE,)
interest_expense_new_long_term_borrowing

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
company,,,,,,,,,,,,
ZMF,"1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89"
ZAM,"1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67"
NBS,"3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17"
ZB,"5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33"


In [ ]:
def aggregate_interest_expense_short_term_borrowing(
    interest_expense_existing_short_term_borrowing,
    interest_expense_new_short_term_borrowing,
):
    return interest_expense_existing_short_term_borrowing.add(
        interest_expense_new_short_term_borrowing, fill_value=0
    )


def aggregate_interest_expense_long_term_borrowing(
    interest_expense_existing_long_term_borrowing,
    interest_expense_new_long_term_borrowing,
):
    return interest_expense_existing_long_term_borrowing.add(
        interest_expense_new_long_term_borrowing, fill_value=0
    )


def aggregate_interest_expense_on_borrowing(
    interest_expense_short_term_borrowing, interest_expense_long_term_borrowing
):
    return interest_expense_short_term_borrowing.add(
        interest_expense_long_term_borrowing, fill_value=0
    )

In [ ]:
interest_expense_short_term_borrowing = aggregate_interest_expense_short_term_borrowing(
    interest_expense_existing_short_term_borrowing,
    interest_expense_new_short_term_borrowing,
)
interest_expense_short_term_borrowing

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
company,,,,,,,,,,,,
Beramasamba,325.00,325.00,325.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Dandemutande,250.00,250.00,250.00,250.00,250.00,250.00,250.00,0.00,0.00,0.00,0.00,0.00
FCB,"1,805.74","1,805.74","1,805.74","1,805.74","1,805.74",0.00,0.00,0.00,0.00,0.00,0.00,0.00
Grandre,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00",0.00,0.00,0.00,0.00,0.00
NBS,"7,708.33","7,708.33","7,708.33","7,708.33","7,708.33","7,708.33","7,708.33","7,708.33","7,708.33","7,708.33","7,708.33","7,708.33"
NMB,"8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00",0.00
Nurture,"10,416.67","10,416.67","10,416.67","10,416.67","10,416.67","10,416.67","10,416.67","10,416.67",0.00,0.00,0.00,0.00
ZAM,"8,333.33","8,333.33","8,333.33","8,333.33","8,333.33","8,333.33","8,333.33","8,333.33","8,333.33","8,333.33","8,333.33","8,333.33"
ZB,"8,750.00","8,750.00","8,750.00","8,750.00","8,750.00","8,750.00","8,750.00","8,750.00","8,750.00","8,750.00","8,750.00","8,750.00"


In [ ]:
interest_expense_long_term_borrowing = aggregate_interest_expense_long_term_borrowing(
    interest_expense_existing_long_term_borrowing,
    interest_expense_new_long_term_borrowing,
)
interest_expense_long_term_borrowing

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
company,,,,,,,,,,,,
Grandre,266.67,266.67,266.67,266.67,266.67,266.67,266.67,266.67,266.67,266.67,266.67,266.67
Masawara,5.61,5.61,5.61,5.61,5.61,5.61,5.61,5.61,5.61,5.61,5.61,5.61
Minerva,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33
NBS,"3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17","3,854.17"
ZAM,"1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67","1,666.67"
ZB,"5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33","5,833.33"
ZMF,"1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89","1,888.89"
Zam-Cargo,505.56,505.56,505.56,505.56,505.56,505.56,505.56,505.56,505.56,505.56,505.56,505.56
Zimnat Lion Insurance,92.80,92.80,92.80,92.80,92.80,92.80,92.80,92.80,92.80,92.80,92.80,92.80


In [ ]:
interest_expense = aggregate_interest_expense_on_borrowing(interest_expense_short_term_borrowing, interest_expense_long_term_borrowing)
interest_expense

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
company,,,,,,,,,,,,
Beramasamba,325.00,325.00,325.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Dandemutande,250.00,250.00,250.00,250.00,250.00,250.00,250.00,0.00,0.00,0.00,0.00,0.00
FCB,"1,805.74","1,805.74","1,805.74","1,805.74","1,805.74",0.00,0.00,0.00,0.00,0.00,0.00,0.00
Grandre,"1,266.67","1,266.67","1,266.67","1,266.67","1,266.67","1,266.67","1,266.67",266.67,266.67,266.67,266.67,266.67
Masawara,5.61,5.61,5.61,5.61,5.61,5.61,5.61,5.61,5.61,5.61,5.61,5.61
Minerva,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33,33.33
NBS,"11,562.50","11,562.50","11,562.50","11,562.50","11,562.50","11,562.50","11,562.50","11,562.50","11,562.50","11,562.50","11,562.50","11,562.50"
NMB,"8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00","8,250.00",0.00
Nurture,"10,416.67","10,416.67","10,416.67","10,416.67","10,416.67","10,416.67","10,416.67","10,416.67",0.00,0.00,0.00,0.00


In [ ]:
interest_expense.sum()

Jan-2023   67,811.10
Feb-2023   67,811.10
Mar-2023   67,811.10
Apr-2023   67,486.10
May-2023   67,486.10
Jun-2023   65,680.36
Jul-2023   65,680.36
Aug-2023   64,430.36
Sep-2023   54,013.69
Oct-2023   54,013.69
Nov-2023   53,430.36
Dec-2023   45,180.36
dtype: float64

In [ ]:
income_statement.loc['Finance Costs'] = interest_expense.sum()

In [ ]:
income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [ ]:
income_statement = aggregate_finance_costs(income_statement)
income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [ ]:
income_statement = calculate_profit_before_tax(income_statement)
income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [ ]:
parameters.loc['TAX_RATE']

2023-01   0.25
2023-02   0.25
2023-03   0.25
2023-04   0.25
2023-05   0.25
2023-06   0.25
2023-07   0.25
2023-08   0.25
2023-09   0.25
2023-10   0.25
2023-11   0.25
2023-12   0.25
Freq: M, Name: TAX_RATE, dtype: float64

In [ ]:
income_statement = calculate_tax(income_statement=income_statement,tax_rate=parameters.loc['TAX_RATE'])
income_statement

c:\Users\HP\anaconda3\Lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in maximum
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [ ]:
def calculate_profit_or_loss_for_period(income_statement):
    income_statement.loc['PROFIT/(LOSS) FOR PERIOD'] = income_statement.loc['PROFIT / (LOSS) BEFORE TAX'] - income_statement.loc['Taxation'] - income_statement.loc['2% Taxation']
    return income_statement
    

In [ ]:

income_statement = calculate_profit_or_loss_for_period(income_statement)
income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


In [ ]:
variable_inputs_income_statement

,Training,Travel Costs,Entertainment,Marketing Costs,Group Marketing Costs,Rental Costs,Subscriptions,Insurance,Repairs And Maintenance,Utilities,Stationery,Admin Costs,IT Costs,Masawara Mgt Fee,Fines And Penalties,Auditors Remuneration,Directors Fees,Consultancy Fees,Telephones,Cellphones,Internet,Courier,Fuel,Motor Vehicle Maintenance Costs,Bank Charges,Zimnat Grp Shared Costs Recovery Exp
Aug-22,6028,6484,6678,8157,9891,7038,6282,5663,8763,6636,9097,6733,9496,6930,8397,6986,5488,6802,8643,9090,9499,6648,7557,8777,5160,6218
Sep-22,5502,5863,8242,7625,8627,8302,5709,6998,6853,8696,9737,9931,5766,8582,7511,5146,7976,9061,9976,7056,8191,6445,5098,7869,9493,9496
Oct-22,9493,7790,9636,7729,6363,7237,9744,8445,9659,7999,5854,8510,9389,5608,6794,8219,6959,8369,7049,8170,6696,9199,7200,8234,6300,9735
Nov-22,5870,5563,6059,6597,6981,6306,9413,8743,6291,8152,8474,5202,7327,8272,5659,7911,7385,5262,8723,6682,8991,8923,7961,8791,9911,8104
Dec-22,9488,9191,6722,8060,6663,9029,8748,6495,8581,5698,6707,8255,7931,6147,7811,6734,7919,5623,9548,7255,9723,5253,9973,5956,8987,7735
Jan-23,5206,6757,8314,7693,6529,7675,9779,8304,8457,7160,7777,9218,5197,9282,6369,6843,9736,6016,8108,6154,9682,8183,9433,9067,6527,9555


In [ ]:
details_of_existing_assets

,asset,depreciation,net_book_value,remaining_useful_life,cost,useful_life_when_purchased,method
0,Vehicles,0.20,10000,5,20000,10,straight_line
1,Buildings,0.10,50000,20,80000,25,reducing_balance


In [ ]:
# details_of_existing_assets.to_csv("./data_templates/details_of_existing_assets.csv", index=False)
# details_of_new_assets.to_csv("./data_templates/details_of_new_assets.csv", index=False)
# static_inputs_income_statement.to_csv("./data_templates/static_inputs_income_statement.csv")
# variable_inputs_income_statement.to_csv("./data_templates/variable_inputs_income_statement.csv")
# details_of_existing_long_term_borrowing.to_csv("./data_templates/details_of_existing_long_term_borrowing.csv", index=False)
# details_of_existing_short_term_borrowing.to_csv("./data_templates/details_of_existing_short_term_borrowing.csv", index=False)
# details_of_new_long_term_borrowing.to_csv("./data_templates/details_of_new_long_term_borrowing.csv", index=False)
# details_of_new_short_term_borrowing.to_csv("./data_templates/details_of_new_short_term_borrowing.csv", index=False)

In [ ]:
# pd.Series(income_statement.index).to_csv("./data_templates/income_statement_index.csv", index = False)

## **Direct Cashflow**

In [ ]:
direct_cashflow = pd.DataFrame(
    index=[
        "CASH INFLOWS",
        "Receipts From Trade Receivables",
        'Capital Repayment', 
        "Interest Income",
        "Other Income",
        "Issue Of Shares",
        "Short Term Borrowing",
        "Long Term Borrowing",
        "Total Cash Inflows",
        "CASH OUTFLOWS",
        "Disbursements",
        "Operating Expenses",
        "Capital Expenses",
        "Interest Expense",
        'Dividend Paid', 
        'Capital Repayment On Borrowings', 
        'Tax Paid', 
        'Total Cash Outflows'
        , 'Net Increase/Decrease In Cash', 
        'Opening Balance', 
        'Closing Balance'
    ], 
    columns= generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)

)
direct_cashflow.loc[direct_cashflow.index.str.isupper()] = ""
direct_cashflow

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other Income,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Issue Of Shares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Cash Inflows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CASH OUTFLOWS,,,,,,,,,,,,


In [ ]:
def generate_direct_cashflow_template(
    direct_cashflow_index, valuation_date, months_to_forecase
):
    direct_cashflow = pd.DataFrame(
        index=[
            "CASH INFLOWS",
            "Receipts From Trade Receivables",
            "Capital Repayment",
            "Interest Income",
            "Other Income",
            "Issue Of Shares",
            "Short Term Borrowing",
            "Long Term Borrowing",
            "Total Cash Inflows",
            "CASH OUTFLOWS",
            "Disbursements",
            "Operating Expenses",
            "Capital Expenses",
            "Interest Expense",
            "Dividend Paid",
            "Capital Repayment On Borrowings",
            "Tax Paid",
            "Total Cash Outflows",
            "Net Increase/Decrease In Cash",
            "Opening Balance",
            "Closing Balance",
        ],
        columns=generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST),
    )
    direct_cashflow.loc[direct_cashflow.index.str.isupper()] = ""
    return direct_cashflow

In [ ]:
disbursements = add_series([consumer_pvt_disbursements, consumer_ssb_disbursements, sme_disbursements, b2b_disbursements])
disbursements.index = disbursements.index.to_timestamp().strftime("%b-%Y")
disbursements

Jan-2023     478,000.00
Feb-2023     577,000.00
Mar-2023     670,000.00
Apr-2023     570,000.00
May-2023     574,000.00
Jun-2023     646,000.00
Jul-2023     731,000.00
Aug-2023     927,500.00
Sep-2023   1,055,000.00
Oct-2023   1,345,000.00
Nov-2023   1,480,000.00
Dec-2023   1,480,000.00
dtype: float64

In [ ]:
## Cash Inflows
direct_cashflow.loc['Receipts From Trade Receivables'] = receipts_from_trade_receivables
direct_cashflow.loc['Interest Income'] = income_statement.loc['Interest Income']
direct_cashflow.loc['Other Income'] = income_statement.loc['Other Income']

## Cash Outflows
direct_cashflow.loc['Disbursements'] = disbursements
direct_cashflow.loc['Interest Expense'] = income_statement.loc['Finance Costs']

In [ ]:
provisions = income_statement.loc[income_statement.index.str.contains("provision", case=False)].sum()
income_statement.loc['Depreciation']

Jan-2023   14,500.00
Feb-2023   12,475.00
Mar-2023   10,786.25
Apr-2023    9,368.69
May-2023    8,171.22
Jun-2023    5,904.90
Jul-2023    5,314.41
Aug-2023    4,782.97
Sep-2023    4,304.67
Oct-2023    3,874.20
Nov-2023    3,486.78
Dec-2023    3,138.11
Name: Depreciation, dtype: object

In [ ]:
operating_expenses = income_statement.loc['TOTAL EXPENSES'] - provisions - income_statement.loc['Depreciation']
operating_expenses

Jan-2023   299,404.18
Feb-2023   299,078.04
Mar-2023   304,602.20
Apr-2023   325,184.80
May-2023   321,003.48
Jun-2023   332,025.96
Jul-2023   336,694.64
Aug-2023   346,326.86
Sep-2023   359,085.76
Oct-2023   355,671.39
Nov-2023   374,455.79
Dec-2023   361,106.03
dtype: object

In [ ]:
def calculate_operating_expenses(income_statement:pd.DataFrame): 
    provisions = income_statement.loc[income_statement.index.str.contains("provision", case=False)].sum()
    operating_expenses = income_statement.loc['TOTAL EXPENSES'] - provisions - income_statement.loc['Depreciation']
    return operating_expenses
    

In [ ]:
direct_cashflow.loc['Operating Expenses']  = operating_expenses
direct_cashflow

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Issue Of Shares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Cash Inflows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CASH OUTFLOWS,,,,,,,,,,,,


In [ ]:
def convert_to_datetime(date: pd.Series):
    try:
        date = pd.to_datetime(date, format="%d/%m/%Y")
    except:
        date = pd.to_datetime(date, format="%m/%d/%Y")

    return date

In [ ]:
def calculate_capital_expenses(details_of_new_assets: pd.DataFrame): 
    capital_expenses = details_of_new_assets[['cost', 'purchase_date']]
    capital_expenses = capital_expenses.assign(purchase_date = convert_to_datetime(capital_expenses['purchase_date']))
    capital_expenses['purchase_date'] = capital_expenses['purchase_date'].dt.to_period("M").dt.strftime("%b-%Y")
    capital_expenses = capital_expenses.groupby("purchase_date")['cost'].sum().reindex(generate_columns(VALUATION_DATE,MONTHS_TO_FORECAST),fill_value=0)
    return capital_expenses


In [ ]:
capital_expenses = calculate_capital_expenses(details_of_new_assets)

In [ ]:
direct_cashflow.loc['Capital Expenses'] = capital_expenses

In [ ]:
direct_cashflow

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Capital Repayment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Issue Of Shares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Long Term Borrowing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Cash Inflows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CASH OUTFLOWS,,,,,,,,,,,,


In [ ]:
details_of_new_short_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,12,12,01/07/2023,straight_line
1,ZAM,400000,0.25,12,12,01/07/2023,reducing_balance
2,NBS,500000,0.18,12,12,01/09/2023,straight_line
3,ZB,500000,0.21,12,12,01/10/2023,straight_line


In [ ]:
details_of_new_long_term_borrowing

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,36,36,01/08/2023,straight_line
1,ZAM,400000,0.25,60,60,01/09/2023,straight_line
2,NBS,500000,0.18,24,24,01/10/2023,straight_line
3,ZB,500000,0.21,18,18,01/10/2023,straight_line


In [ ]:
def calculate_direct_cashflow_borrowing(details_of_new_borrowing: pd.DataFrame): 
    direct_cashflow_borrowing = details_of_new_borrowing[['principal', 'loan_start_date']]
    direct_cashflow_borrowing = direct_cashflow_borrowing.assign(loan_start_date = convert_to_datetime(direct_cashflow_borrowing['loan_start_date']))
    direct_cashflow_borrowing['loan_start_date'] = direct_cashflow_borrowing['loan_start_date'].dt.to_period("M").dt.strftime("%b-%Y")
    direct_cashflow_borrowing = direct_cashflow_borrowing.groupby("loan_start_date")['principal'].sum().reindex(generate_columns(VALUATION_DATE,MONTHS_TO_FORECAST),fill_value=0)
    return direct_cashflow_borrowing


In [ ]:
short_term_borrowing = calculate_direct_cashflow_borrowing(details_of_new_short_term_borrowing)
short_term_borrowing

Jan-2023         0
Feb-2023         0
Mar-2023         0
Apr-2023         0
May-2023         0
Jun-2023         0
Jul-2023    600000
Aug-2023         0
Sep-2023    500000
Oct-2023    500000
Nov-2023         0
Dec-2023         0
Name: principal, dtype: int64

In [ ]:
long_term_borrowing = calculate_direct_cashflow_borrowing(details_of_new_long_term_borrowing)
long_term_borrowing

Jan-2023          0
Feb-2023          0
Mar-2023          0
Apr-2023          0
May-2023          0
Jun-2023          0
Jul-2023          0
Aug-2023     200000
Sep-2023     400000
Oct-2023    1000000
Nov-2023          0
Dec-2023          0
Name: principal, dtype: int64

In [ ]:
direct_cashflow.loc['Short Term Borrowing'] = short_term_borrowing
direct_cashflow.loc['Long Term Borrowing'] = long_term_borrowing

## **Capital Repayment**

In [ ]:
sme_loan_schedules.keys()

dict_keys(['interest', 'capital_repayments', 'outstanding_at_start'])

In [ ]:
sme_loan_schedules["capital_repayments"].sum()

Feb-2023    47,251.18
Mar-2023   108,441.46
Apr-2023   181,744.98
May-2023   246,261.93
Jun-2023   312,182.88
Jul-2023   391,453.99
Aug-2023   402,317.32
Sep-2023   420,891.75
Oct-2023   444,152.19
Nov-2023   534,836.21
Dec-2023   659,999.53
Jan-2024   784,835.69
dtype: float64

In [ ]:
capital_repayments_new_disbursements = add_series(
    [
        sme_loan_schedules["capital_repayments"].sum(),
        consumer_pvt_loan_schedules["capital_repayments"].sum(),
        consumer_ssb_loan_schedules["capital_repayments"].sum(),
        b2b_loan_schedules["capital_repayments"].sum(),
    ]
)
capital_repayments_new_disbursements

Feb-2023    56,172.91
Mar-2023   129,167.71
Apr-2023   218,801.00
May-2023   297,231.35
Jun-2023   380,312.28
Jul-2023   479,190.67
Aug-2023   513,571.01
Sep-2023   556,986.58
Oct-2023   608,453.22
Nov-2023   710,699.61
Dec-2023   842,349.66
Jan-2024   966,750.42
dtype: float64

In [ ]:
capital_repayment_on_existing_loans = existing_loans_schedules['interest'].sum()
capital_repayment_on_existing_loans

Jan-2023   290.78
Feb-2023   283.97
Mar-2023   277.09
Apr-2023   270.13
May-2023   263.09
Jun-2023   255.98
Jul-2023   248.78
Aug-2023   241.51
Sep-2023   234.16
Oct-2023   226.73
Nov-2023   219.21
Dec-2023     0.00
dtype: float64

In [ ]:
capital_repayment = add_series([capital_repayment_on_existing_loans, capital_repayments_new_disbursements])
capital_repayment

Jan-2023       290.78
Feb-2023    56,456.88
Mar-2023   129,444.80
Apr-2023   219,071.13
May-2023   297,494.44
Jun-2023   380,568.25
Jul-2023   479,439.45
Aug-2023   513,812.52
Sep-2023   557,220.74
Oct-2023   608,679.95
Nov-2023   710,918.81
Dec-2023   842,349.66
Jan-2024   966,750.42
dtype: float64

In [ ]:
direct_cashflow.loc['Capital Repayment'] = capital_repayment

In [ ]:
direct_cashflow

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Capital Repayment,290.78,"56,456.88","129,444.80","219,071.13","297,494.44","380,568.25","479,439.45","513,812.52","557,220.74","608,679.95","710,918.81","842,349.66"
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Issue Of Shares,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Short Term Borrowing,0,0,0,0,0,0,600000,0,500000,500000,0,0
Long Term Borrowing,0,0,0,0,0,0,0,200000,400000,1000000,0,0
Total Cash Inflows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CASH OUTFLOWS,,,,,,,,,,,,


In [ ]:
def insert_issue_of_shares(direct_cashflow: pd.DataFrame, parameters:pd.DataFrame): 
    direct_cashflow.loc['Issue Of Shares'] = parameters.loc['ISSUE_OF_SHARES']
    return direct_cashflow

    

In [ ]:
direct_cashflow = insert_issue_of_shares(direct_cashflow, parameters)
direct_cashflow

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Capital Repayment,290.78,"56,456.88","129,444.80","219,071.13","297,494.44","380,568.25","479,439.45","513,812.52","557,220.74","608,679.95","710,918.81","842,349.66"
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Issue Of Shares,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
Short Term Borrowing,0,0,0,0,0,0,600000,0,500000,500000,0,0
Long Term Borrowing,0,0,0,0,0,0,0,200000,400000,1000000,0,0
Total Cash Inflows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CASH OUTFLOWS,,,,,,,,,,,,


In [ ]:
details_of_existing_long_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Lion Insurance,92800,0.06,60,60,straight_line
1,Zimnat Lion Insurance,50000,0.06,12,12,straight_line
2,Grandre,160000,0.06,36,36,straight_line
3,Minerva,15000,0.08,36,36,straight_line
4,Masawara,5613,0.06,60,60,straight_line
5,Zam-Cargo,364000,0.05,36,36,straight_line


In [ ]:
details_of_existing_short_term_borrowing

,company,principal,interest_rate,remaining_loan_term,loan_term,method
0,Zimnat Life,50000,0.14,10,12,straight_line
1,Zimnat Life,30000,0.13,12,12,straight_line
2,Grandre,100000,0.12,7,12,straight_line
3,FCB,120383,0.18,5,12,straight_line
4,Nurture,500000,0.25,8,12,straight_line
5,NMB,450000,0.22,11,12,straight_line
6,Beramasamba,30000,0.13,3,12,straight_line
7,Dandemutande,25000,0.12,7,12,reducing_balance


In [ ]:
def aggregate_existing_short_and_long_term_borrowing(details_of_existing_long_term_borrowing, details_of_existing_short_term_borrowing): 
    details_of_existing_borrowings = pd.concat([details_of_existing_long_term_borrowing, details_of_existing_short_term_borrowing])
    details_of_existing_borrowings = details_of_existing_borrowings.assign(loan_end_date = pd.Period(VALUATION_DATE) + details_of_existing_borrowings['remaining_loan_term'] )
    return details_of_existing_borrowings

In [ ]:
details_of_existing_borrowings = aggregate_existing_short_and_long_term_borrowing(details_of_existing_long_term_borrowing, details_of_existing_short_term_borrowing)
details_of_existing_borrowings

,company,principal,interest_rate,remaining_loan_term,loan_term,method,loan_end_date
0,Zimnat Lion Insurance,92800,0.06,60,60,straight_line,2028-01
1,Zimnat Lion Insurance,50000,0.06,12,12,straight_line,2024-01
2,Grandre,160000,0.06,36,36,straight_line,2026-01
3,Minerva,15000,0.08,36,36,straight_line,2026-01
4,Masawara,5613,0.06,60,60,straight_line,2028-01
5,Zam-Cargo,364000,0.05,36,36,straight_line,2026-01
0,Zimnat Life,50000,0.14,10,12,straight_line,2023-11
1,Zimnat Life,30000,0.13,12,12,straight_line,2024-01
2,Grandre,100000,0.12,7,12,straight_line,2023-08
3,FCB,120383,0.18,5,12,straight_line,2023-06


In [ ]:

def aggregate_new_short_and_long_term_borrowing(
    details_of_new_long_term_borrowing,
    details_of_new_short_term_borrowing,
    valuation_date,
):
    
    details_of_new_borrowings = pd.concat([details_of_new_long_term_borrowing, details_of_new_short_term_borrowing])
    return details_of_new_borrowings

In [ ]:
details_of_new_borrowings = pd.concat([details_of_new_long_term_borrowing, details_of_new_short_term_borrowing])
details_of_new_borrowings

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method
0,ZMF,200000,0.34,36,36,01/08/2023,straight_line
1,ZAM,400000,0.25,60,60,01/09/2023,straight_line
2,NBS,500000,0.18,24,24,01/10/2023,straight_line
3,ZB,500000,0.21,18,18,01/10/2023,straight_line
0,ZMF,200000,0.34,12,12,01/07/2023,straight_line
1,ZAM,400000,0.25,12,12,01/07/2023,reducing_balance
2,NBS,500000,0.18,12,12,01/09/2023,straight_line
3,ZB,500000,0.21,12,12,01/10/2023,straight_line


In [ ]:
def calculate_loan_end_date_on_new_borrowing(details_of_new_borrowings: pd.DataFrame): 
    details_of_new_borrowings = details_of_new_borrowings.assign(loan_start_date = convert_to_datetime(details_of_new_borrowings.loan_start_date))
    details_of_new_borrowings = details_of_new_borrowings.assign(loan_start_date = details_of_new_borrowings.loan_start_date.dt.to_period("M"))
    details_of_new_borrowings = details_of_new_borrowings.assign(loan_end_date = details_of_new_borrowings.loan_start_date + details_of_new_borrowings.remaining_loan_term)
    return details_of_new_borrowings

In [ ]:
details_of_new_borrowings = calculate_loan_end_date_on_new_borrowing(details_of_new_borrowings)
details_of_new_borrowings

,company,principal,interest_rate,loan_term,remaining_loan_term,loan_start_date,method,loan_end_date
0,ZMF,200000,0.34,36,36,2023-08,straight_line,2026-08
1,ZAM,400000,0.25,60,60,2023-09,straight_line,2028-09
2,NBS,500000,0.18,24,24,2023-10,straight_line,2025-10
3,ZB,500000,0.21,18,18,2023-10,straight_line,2025-04
0,ZMF,200000,0.34,12,12,2023-07,straight_line,2024-07
1,ZAM,400000,0.25,12,12,2023-07,reducing_balance,2024-07
2,NBS,500000,0.18,12,12,2023-09,straight_line,2024-09
3,ZB,500000,0.21,12,12,2023-10,straight_line,2024-10


In [ ]:
def calculate_direct_cashflow_capital_repayment(details_of_new_borrowing: pd.DataFrame): 
    direct_cashflow_capital_repayment = details_of_new_borrowing[['principal', 'loan_end_date']]
    direct_cashflow_capital_repayment = direct_cashflow_capital_repayment.groupby("loan_end_date")['principal'].sum().reindex(generate_columns(VALUATION_DATE,MONTHS_TO_FORECAST),fill_value=0)
    return direct_cashflow_capital_repayment

In [ ]:
capital_repayment_on_new_borrowings = calculate_direct_cashflow_capital_repayment(details_of_new_borrowings)
capital_repayment_on_new_borrowings

Jan-2023    0
Feb-2023    0
Mar-2023    0
Apr-2023    0
May-2023    0
Jun-2023    0
Jul-2023    0
Aug-2023    0
Sep-2023    0
Oct-2023    0
Nov-2023    0
Dec-2023    0
Name: principal, dtype: int64

In [ ]:
capital_repayment_on_existing_borrowings = calculate_direct_cashflow_capital_repayment(details_of_existing_borrowings)
capital_repayment_on_existing_borrowings

Jan-2023         0
Feb-2023         0
Mar-2023         0
Apr-2023     30000
May-2023         0
Jun-2023    120383
Jul-2023         0
Aug-2023    125000
Sep-2023    500000
Oct-2023         0
Nov-2023     50000
Dec-2023    450000
Name: principal, dtype: int64

In [ ]:
capital_repayment_on_borrowings  = capital_repayment_on_new_borrowings + capital_repayment_on_existing_borrowings
capital_repayment_on_borrowings

Jan-2023         0
Feb-2023         0
Mar-2023         0
Apr-2023     30000
May-2023         0
Jun-2023    120383
Jul-2023         0
Aug-2023    125000
Sep-2023    500000
Oct-2023         0
Nov-2023     50000
Dec-2023    450000
Name: principal, dtype: int64

In [ ]:
direct_cashflow.loc['Capital Repayment On Borrowings'] = capital_repayment_on_borrowings

In [ ]:
direct_cashflow.loc['Tax Paid'] = parameters.loc['TAX_PAID']
direct_cashflow.loc['Dividend Paid'] = parameters.loc['DIVIDEND_PAID']

In [ ]:
direct_cashflow.loc['Total Cash Inflows'] = direct_cashflow.loc['Receipts From Trade Receivables':"Long Term Borrowing"].sum()
direct_cashflow.loc['Total Cash Outflows'] = direct_cashflow.loc["Disbursements":"Tax Paid"].sum()

In [ ]:
direct_cashflow.loc['Net Increase/Decrease In Cash'] = direct_cashflow.loc['Total Cash Inflows'] - direct_cashflow.loc['Total Cash Outflows']

In [ ]:
direct_cashflow

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Capital Repayment,290.78,"56,456.88","129,444.80","219,071.13","297,494.44","380,568.25","479,439.45","513,812.52","557,220.74","608,679.95","710,918.81","842,349.66"
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Issue Of Shares,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
Short Term Borrowing,0,0,0,0,0,0,600000,0,500000,500000,0,0
Long Term Borrowing,0,0,0,0,0,0,0,200000,400000,1000000,0,0
Total Cash Inflows,"195,669.68","296,952.85","416,333.30","500,163.29","585,291.34","690,867.70","1,412,168.74","1,092,076.47","1,832,867.82","2,500,555.32","1,040,617.40","1,058,672.56"
CASH OUTFLOWS,,,,,,,,,,,,


In [ ]:
direct_cashflow.loc['Opening Balance',direct_cashflow.columns[0]] = opening_balances.loc['CASH_ON_HAND'].iat[0]

In [ ]:
direct_cashflow

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Capital Repayment,290.78,"56,456.88","129,444.80","219,071.13","297,494.44","380,568.25","479,439.45","513,812.52","557,220.74","608,679.95","710,918.81","842,349.66"
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Issue Of Shares,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
Short Term Borrowing,0,0,0,0,0,0,600000,0,500000,500000,0,0
Long Term Borrowing,0,0,0,0,0,0,0,200000,400000,1000000,0,0
Total Cash Inflows,"195,669.68","296,952.85","416,333.30","500,163.29","585,291.34","690,867.70","1,412,168.74","1,092,076.47","1,832,867.82","2,500,555.32","1,040,617.40","1,058,672.56"
CASH OUTFLOWS,,,,,,,,,,,,


In [ ]:
direct_cashflow.columns = pd.PeriodIndex(direct_cashflow.columns, freq="M")

In [ ]:
for period in direct_cashflow.columns:
    direct_cashflow.loc["Closing Balance", period] = direct_cashflow.loc[
        "Net Increase/Decrease In Cash":"Opening Balance", period
    ].sum()
    if period == direct_cashflow.columns[-1]: 
        break
    direct_cashflow.loc['Opening Balance',period + 1] = direct_cashflow.loc['Closing Balance',period]

In [ ]:
def calculate_opening_and_closing_balances_for_direct_cashflows(
    direct_cashflow: pd.DataFrame, cash_on_hand_opening_balance: float
):
    direct_cashflow.loc[
        "Opening Balance", direct_cashflow.columns[0]
    ] = cash_on_hand_opening_balance

    for period in direct_cashflow.columns:
        direct_cashflow.loc["Closing Balance", period] = direct_cashflow.loc[
            "Net Increase/Decrease In Cash":"Opening Balance", period
        ].sum()
        if period == direct_cashflow.columns[-1]:
            break
        direct_cashflow.loc["Opening Balance", period + 1] = direct_cashflow.loc[
            "Closing Balance", period
        ]
        
    return direct_cashflow

In [ ]:
direct_cashflow

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
CASH INFLOWS,,,,,,,,,,,,
Receipts From Trade Receivables,"1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00","1,000.00"
Capital Repayment,290.78,"56,456.88","129,444.80","219,071.13","297,494.44","380,568.25","479,439.45","513,812.52","557,220.74","608,679.95","710,918.81","842,349.66"
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Issue Of Shares,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00
Short Term Borrowing,0,0,0,0,0,0,600000,0,500000,500000,0,0
Long Term Borrowing,0,0,0,0,0,0,0,200000,400000,1000000,0,0
Total Cash Inflows,"195,669.68","296,952.85","416,333.30","500,163.29","585,291.34","690,867.70","1,412,168.74","1,092,076.47","1,832,867.82","2,500,555.32","1,040,617.40","1,058,672.56"
CASH OUTFLOWS,,,,,,,,,,,,


In [ ]:
income_statement.loc['2% Taxation']  = direct_cashflow.loc['Total Cash Outflows']*IMTT
income_statement

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
INCOME STATEMENT,,,,,,,,,,,,
Interest Income,"189,400.85","228,667.56","268,375.14","257,174.38","258,655.84","275,190.55","295,104.59","336,792.80","329,521.29","336,211.61","262,711.85","138,925.00"
Other Income,"4,978.06","10,828.41","17,513.37","22,917.78","28,141.06","34,108.89","36,624.70","40,471.16","45,125.79","54,563.77","65,986.74","76,397.89"
Total Revenue,"194,378.90","239,495.97","285,888.51","280,092.16","286,796.90","309,299.45","331,729.28","377,263.95","374,647.08","390,775.37","328,698.59","215,322.89"
MANAGEMENT EXPENSES,,,,,,,,,,,,
STAFF COSTS,,,,,,,,,,,,
Salaries,"27,200.00","29,420.00","32,300.00","32,200.00","34,340.00","35,780.00","38,850.00","41,975.00","44,150.00","47,450.00","48,800.00","48,800.00"
Pensions & Statutory Contributions,"3,089.92","3,342.11","3,669.28","3,657.92","3,901.02","4,064.61","4,413.36","4,768.36","5,015.44","5,390.32","5,543.68","5,543.68"
Training,"7,069.79","7,211.19","7,355.41","7,502.52","7,652.57","7,805.62","7,961.73","8,120.97","8,283.39","8,449.05","8,618.03","8,790.40"
Bonus Provision,14297,10995,17629,19467,11016,17869,16439,17892,16863,17916,18529,10878


## **Loan Book**


In [ ]:
loan_book = pd.DataFrame(
    index=[
        "Opening Balance",
        "New Disbursements",
        "Capital Repayments",
        "Closing Balance",
    ]
    , columns=generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
)
loan_book

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def generate_loan_book_template(valuation_date:str, months_to_forecast:int): 
    loan_book = pd.DataFrame(
    index=[
        "Opening Balance",
        "New Disbursements",
        "Capital Repayments",
        "Closing Balance",
    ]
    , columns=generate_columns(VALUATION_DATE, MONTHS_TO_FORECAST)
    )
    return loan_book


In [ ]:
loan_book.loc['Opening Balance', loan_book.columns[0]] =  float(opening_balances.loc['LOAN_BOOK'].iat[0])

In [ ]:
loan_book

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,"2,800,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Capital Repayments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
loan_book.loc['New Disbursements'] = disbursements
loan_book

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,"2,800,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,"478,000.00","577,000.00","670,000.00","570,000.00","574,000.00","646,000.00","731,000.00","927,500.00","1,055,000.00","1,345,000.00","1,480,000.00","1,480,000.00"
Capital Repayments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
loan_book.loc['Capital Repayments'] = capital_repayment
loan_book

,Jan-2023,Feb-2023,Mar-2023,Apr-2023,May-2023,Jun-2023,Jul-2023,Aug-2023,Sep-2023,Oct-2023,Nov-2023,Dec-2023
Opening Balance,"2,800,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New Disbursements,"478,000.00","577,000.00","670,000.00","570,000.00","574,000.00","646,000.00","731,000.00","927,500.00","1,055,000.00","1,345,000.00","1,480,000.00","1,480,000.00"
Capital Repayments,290.78,"56,456.88","129,444.80","219,071.13","297,494.44","380,568.25","479,439.45","513,812.52","557,220.74","608,679.95","710,918.81","842,349.66"
Closing Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def insert_loan_book_values(
    loan_book: pd.DataFrame, 
    opening_balance_on_loan_book: float,
    capital_repayment: pd.Series,
    disbursements: pd.Series
):
    loan_book.loc[
        "Opening Balance", loan_book.columns[0]
    ] = opening_balance_on_loan_book
    loan_book.loc["Capital Repayments"] = capital_repayment
    loan_book.loc["New Disbursements"] = disbursements

    return loan_book

In [ ]:
loan_book.columns = pd.period_range(VALUATION_DATE, periods=MONTHS_TO_FORECAST, freq="M")

In [1]:
for period in loan_book.columns:
    loan_book.loc["Closing Balance", period] = loan_book.loc[
        "Opening Balance":"Capital Repayments", period
    ].sum()
    if period == loan_book.columns[-1]: 
        break
    loan_book.loc['Opening Balance',period + 1] = loan_book.loc['Closing Balance',period]
    

NameError: name 'loan_book' is not defined

In [ ]:
def calculate_opening_and_closing_balances_for_loan_book(loan_book: pd.DataFrame):
    direct_cashflow.columns = pd.PeriodIndex(direct_cashflow.columns, freq="M")
    for period in loan_book.columns:
        loan_book.loc["Closing Balance", period] = loan_book.loc[
            "Opening Balance":"Capital Repayments", period
        ].sum()
        if period == loan_book.columns[-1]:
            break
        loan_book.loc["Opening Balance", period + 1] = loan_book.loc[
            "Closing Balance", period
        ]
        return loan_book

In [338]:
loan_book

,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
Opening Balance,"2,800,000.00","3,278,290.78","3,911,747.66","4,711,192.45","5,500,263.58","6,371,758.02","7,398,326.27","8,608,765.73","10,050,078.25","11,662,298.99","13,615,978.94","15,806,897.75"
New Disbursements,"478,000.00","577,000.00","670,000.00","570,000.00","574,000.00","646,000.00","731,000.00","927,500.00","1,055,000.00","1,345,000.00","1,480,000.00","1,480,000.00"
Capital Repayments,290.78,"56,456.88","129,444.80","219,071.13","297,494.44","380,568.25","479,439.45","513,812.52","557,220.74","608,679.95","710,918.81","842,349.66"
Closing Balance,"3,278,290.78","3,911,747.66","4,711,192.45","5,500,263.58","6,371,758.02","7,398,326.27","8,608,765.73","10,050,078.25","11,662,298.99","13,615,978.94","15,806,897.75","18,129,247.42"
